In [181]:
import psd_tools as pt
from psd_tools import PSDImage
from enum import IntFlag, unique, Enum
from strenum import StrEnum
import string
import win32com.client
import comtypes.client
import pandas as pd
import requests as req
import numpy as np
import sys
import os
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import re

from IPython import display

In [154]:
def PrintLayersHierarchy(obj, depth = 0, isVisible = True, printOnlyVisibleLayers = False, visibilityCharacter = "X") :
    indent = '\t' * depth
    visibility = isVisible and obj.visible
     
    if (printOnlyVisibleLayers and (not visibility)) :
        return
    
    if (obj.is_group()) :
        print(f"{indent}-> [{obj.name}] {visibilityCharacter if visibility else ''}")
        for subObj in obj :
            PrintLayersHierarchy(subObj, depth + 1, visibility, printOnlyVisibleLayers, visibilityCharacter)
    else :
            print(f"{indent}-> {obj.name} {visibilityCharacter if visibility else ''}")
            
def SelectFromBoundaries(photoshopApp, boundaries) :
    photoshopApp.ActiveDocument.Selection.Select(((boundaries[0], boundaries[1]), 
                                        (boundaries[0], boundaries[3]), 
                                        (boundaries[2], boundaries[3]), 
                                        (boundaries[2], boundaries[1])))

In [155]:
@unique
class CardType(IntFlag) :
    Error               = -1
    
    Spell               = 1
    SpellNormal         = 11         
    SpellField          = 12
    SpellRitual         = 13    
    SpellEquip          = 14
    SpellQuickPlay      = 15
    SpellContinuous     = 16

    Trap                = 2
    TrapNormal          = 21
    TrapCounter         = 22
    TrapContinuous      = 23

    Monster             = 3
    MonsterXyz          = 31
    MonsterSynchro      = 32
    MonsterFusion       = 33
    MonsterRitual       = 34
    MonsterEffect       = 35
    MonsterNormal       = 36
    MonsterDarkSynchro  = 37
    MonsterLink         = 38

    Token               = 4
    TokenMonster        = 41
    TokenOther          = 42
    
@unique
class Direction(IntFlag) :
    Left    = 0
    Top     = 1
    Right   = 2
    Bottom  = 3
    Middle  = -1
    

@unique
class TextColor(IntFlag) :
    Black   = 1
    White   = 2
    Silver  = 3
    Gold    = 4
    
@unique
class LinkDirections(Enum) :
    Top            = 1
    Bottom         = 2
    Left           = 3
    Right          = 4
    BottomLeft     = 5
    BottomRight    = 6
    TopLeft        = 7
    TopRight       = 8

In [156]:
class LayerNames(StrEnum) :
# Big groups
    SpellTrapGroup                    = "Spell/Trap"
    MonsterGroup                      = "Monster"
    CardNamesGroup                    = "Card Name"
    AttributeGroup                    = "Attributes"
    LocalisedAttributeGroup           = "ATTRIBUTEGROUP" # Must be passed to LayerNames.GetAttributeForLanguage
    LinkTools                         = "Link Tools"
    PendulumTools                     = "Pendulum Tools"
# Effect Text Box :                       
    MonsterRace                       = "Monster Race"
    MonsterEffect                     = "Monster Effect"
    NormalMonsterEffect               = "Monster Description"
    SpellTrapEffect                   = "Spell/Trap Effect"
    TokenEffect                       = "Monster Effect" 
    
    SampleImage                       = "Sample Image"     # <--,  Must be in the
    CurrentArtwork                    = "Current Artwork"  # <--'   same  group !
# Every Frame
    FrameGroup                        = "Textures" # This group contains the 15 next layers
    XyzMonsterFrame                   = "Xyz Texture"
    SynchroMonsterFrame               = "Synchro Texture"
    FusionMonsterFrame                = "Fusion Texture"
    RitualMonsterFrame                = "Ritual Texture"
    LinkMonsterFrame                  = "Link Texture With Text BG"
    EffectMonsterFrame                = "Effect Texture"
    NormalMonsterFrame                = "Normal Texture"
    TokenMonsterFrame                 = "Token Texture"
    SpellFrame                        = "Spell Texture"
    TrapFrame                         = "Trap Texture"
# Anime
    SliferFrame                       = "Slifer Texture"
    ObeliskFrame                      = "Obelisk Texture"
    RaFrame                           = "Ra Texture"
    LegendaryDragonFrame              = "Legendary Dragon Texture"
    DarkSynchroFrame                  = "Dark Synchro Texture"
# S/T TYPES ->
    STTypesGroup                      = "Spell/Trap Types" # This group contains the 6 next layers
    Field                             = "Field"
    Ritual                            = "Ritual"
    Equip                             = "Equip"
    QuickPlay                         = "Quick-Play"
    Continuous                        = "Continuous"
    Counter                           = "Counter"
# S/T : "Carte Magie"/"Carte Piège"   
    NormalSpellCard                   = "Normal Spell"
    SpecialSpellCard                  = "(Type) Spell"
    NormalTrapCard                    = "Normal Trap"
    SpecialTrapCard                   = "(Type) Trap"
    
# ATK/DEF                             
    AtkDefGroup                       = "ATK/DEF"
    ATK                               = "ATK"
    DEF                               = "DEF"
    ATKisQuestionMark                 = "?forATK"
    DEFisQuestionMark                 = "?forDEF"
    
# Metadata
    CardNameBounds                    = "Card Name - Bounds"
    Edition                           = "Edition"
    SerialNumber                      = "Code"
    SetNumberNormal                   = "Set Number (General)"
    SetNumberLink                     = "Set Number (Link)"
    SetNumberPendulum                 = "Set Number (Pendulum)"
    Copyright                         = "Copyright"
    LayerThatMakesTextOnFrameWhite    = "Make Text On Frame White"
    
# Brackets :
    BracketMonsterRace                = ""
    BracketSpellTrapType              = ""
# Levels :
    LevelRankGroup                    = "Level/Rank"
    CurrentLevel                      = "Current Level"
    Levels                            = "Levels"
    Ranks                             = "Ranks"
    NegativeLevels                    = "Negative Levels"
# Link 
    LinkArrowsGroup                   = "Link Arrows"
    LinkTopArrow                      = "Link: Top Arrow"
    LinkBottomArrow                   = "Link: Bottom Arrow"
    LinkLeftArrow                     = "Link: Left Arrow"
    LinkRightArrow                    = "Link: Right Arrow"
    LinkLowerLeftArrow                = "Link: Lower Left Arrow"
    LinkLowerRightArrow               = "Link: Lower Right Arrow"
    LinkUpperLeftArrow                = "Link: Upper Left Arrow"
    LinkUpperRightArrow               = "Link: Upper Right Arrow"
    LinkArrowActive                   = "Active"
    LinkArrowInactive                 = "Inactive"
    
    LinkNumber                        = "Link Number"
# Pendulum
    PendulumMonsterEffect             = "Monster Effect (Pendulum)"
    PendulumNormalMonsterEffect       = "Monster Description (Pendulum)"
    PendulumSpellEffect               = "Pendulum Effect"
    PendulumScaleRed                  = "Scale Number (Red)"
    PendulumScaleBlue                 = "Scale Number (Blue)"
    PendulumMonsterRace               = "Type/Ability (Pendulum)"
    
    PendulumSmallGroup                = "Pendulum (Small)"
    PendulumMediumGroup               = "Pendulum (Medium)"
    PendulumLargeGroup                = "Pendulum (Large)"
    PendulumThingsToHide              = "To Hide for Pendulum Monsters"

    @classmethod
    def GetAttributeForLanguage(self, attributeInEnglish, language = "en") :
        layerNames = None
        
        if (language.lower() == "en") :
            layerNames = {
                "DARK"   : "DARK",
                "DIVINE" : "DIVINE",
                "EARTH"  : "EARTH",
                "FIRE"   : "FIRE",
                "LIGHT"  : "LIGHT",
                "WATER"  : "WATER",
                "WIND"   : "WIND",
                "SPELL"  : "SPELL",
                "TRAP"   : "TRAP",
                "ATTRIBUTEGROUP" : "FR ONLY"
            }
        elif (language.lower() == "fr") :
            layerNames = {
                "DARK"   : "TENEBRES",
                "DIVINE" : "DIVIN",
                "EARTH"  : "TERRE",
                "FIRE"   : "FEU",
                "LIGHT"  : "LUMIERE",
                "WATER"  : "EAU",
                "WIND"   : "VENT",
                "SPELL"  : "SPELL",
                "TRAP"   : "TRAP",
                "ATTRIBUTEGROUP" : "EN ONLY"
            }
            
        if (layerNames == None) :
            print(f"ERROR : LayerNames.GetAttributeForLanguage(attributeInEnglish = {attributeInEnglish}, language = {language}) : Language not supported.")
            return None
        
        attributeInEnglish = attributeInEnglish.strip().upper()
        if (attributeInEnglish not in layerNames.keys()) :
            print(f"ERROR : LayerNames.GetAttributeForLanguage(attributeInEnglish = {attributeInEnglish}, language = {language}) : Attribute not found.")
            return None
        else :
            return layerNames[attributeInEnglish]
        
    @classmethod
    def GetCardNameLayerNameFromColor(self, color = TextColor.Black) :
        CardNamesFromColor = {
            TextColor.Black   : "Card Name (Black)",
            TextColor.White   : "Card Name (White)",
            TextColor.Gold    : "Card Name (Gold)",
            TextColor.Silver  : "Card Name (Silver)"
        }
        
        if (color not in CardNamesFromColor.keys()) :
            print(f"ERROR : LayerNames.GetCardNameLayerNameFromColor(color = {color.name}) : Color not supported.")
            return None
        else :
            return CardNamesFromColor[color]
       
    @classmethod
    def GetFrameLayerNameFromCardType(self, cardType = CardType.MonsterEffect) :
        FrameLayerNames = {
            CardType.Spell               : self.SpellFrame,
            CardType.Trap                : self.TrapFrame,
            
            CardType.MonsterXyz          : self.XyzMonsterFrame,
            CardType.MonsterSynchro      : self.SynchroMonsterFrame,
            CardType.MonsterFusion       : self.FusionMonsterFrame,
            CardType.MonsterRitual       : self.RitualMonsterFrame,
            CardType.MonsterLink         : self.LinkMonsterFrame,
            CardType.MonsterEffect       : self.EffectMonsterFrame,
            CardType.MonsterNormal       : self.NormalMonsterFrame,
            CardType.MonsterDarkSynchro  : self.DarkSynchroFrame,

            CardType.Token               : self.TokenMonsterFrame,
            CardType.TokenMonster        : self.TokenMonsterFrame,
            CardType.TokenOther          : self.TokenMonsterFrame
        }
        
        if ((cardType + 0) in FrameLayerNames.keys()) :
            return FrameLayerNames[cardType]
        
        cardArchetype = cardType//10
        if ((cardArchetype != CardType.Monster) and (cardArchetype in FrameLayerNames.keys())) :
            return FrameLayerNames[cardArchetype]
        
        print(f"ERROR : LayerNames.GetFrameLayerNameFromCardType(cardType = {cardType.name}) : Card type not supported.")
        return None
        
    @classmethod
    def GetSetNumberLayerNameFromCardType(self, cardType = CardType.Error) :
        if (cardType == CardType.Error) :
            return None
        elif (cardType == CardType.MonsterLink) :
            return self.SetNumberLink
        else :
            return self.SetNumberNormal
        
    @classmethod
    def GetCardEffetBoxFromCardType(self, cardType = CardType.Error) :
        cardArchetype = cardType // 10
        if (cardType == CardType.Error) :
            return None
        elif (cardType == CardType.MonsterNormal) :
            return self.NormalMonsterEffect
        elif (cardArchetype == CardType.Monster) :
            return self.MonsterEffect
        elif ((cardArchetype == CardType.Spell) or (cardArchetype == CardType.Trap)) :
            return self.SpellTrapEffect
        elif (cardArchetype == CardType.Token) :
            return self.TokenEffect
        else : 
            print(f"ERROR : LayerNames.GetCardEffetBoxFromCardType(cardType = {cardType.name}) : Card type not supported.")
            return None
              
    @classmethod
    def GetSpellTrapTypeFromCardType(self, cardType = CardType.Error) :
        cardArchetype = cardType // 10
        if (cardArchetype == CardType.Spell) :
            if (cardType == CardType.SpellNormal) :
                return None
            elif (cardType == CardType.SpellContinuous) :
                return self.Continuous
            elif (cardType == CardType.SpellEquip) :
                return self.Equip
            elif (cardType == CardType.SpellField) :
                return self.Field
            elif (cardType == CardType.SpellQuickPlay) :
                return self.QuickPlay
            elif (cardType == CardType.SpellRitual) :
                return self.Ritual
        elif (cardArchetype == CardType.Trap) :
            if (cardType == CardType.TrapNormal) :
                return None
            elif (cardType == CardType.TrapContinuous) :
                return self.Continuous
            elif (cardType == CardType.TrapCounter) :
                return self.Counter
        
        #ELSE
        print(f"ERROR : LayerNames.GetSpellTrapTypeFromCardType(cardType = {cardType.name}) : Card type not supported.")
        return None
    
    @classmethod
    def GetAllTextInFrameFromCardType(self, cardType = CardType.Error) :
        tempResult = []
        tempResult.append(self.GetSetNumberLayerNameFromCardType(cardType))
        tempResult.append(self.Edition)
        tempResult.append(self.SerialNumber)
        tempResult.append(self.Copyright)
        tempResult.append(self.Copyright)
        tempResult.append(" ")
        
        isValid = lambda x : ((x is not None) and (x.strip() != "")) 
        
        result = []
        for layerName in tempResult :
            if (isValid(layerName) and (layerName not in result)) :
                result.append(layerName)
        return result
    
    @classmethod
    def GetLinkArrowLayerNameFromEnum(self, linkArrow = LinkDirections.Bottom) :
        if (linkArrow == LinkDirections.Top) :
            return self.LinkTopArrow
        elif (linkArrow == LinkDirections.Bottom) :
            return self.LinkBottomArrow
        elif (linkArrow == LinkDirections.Left) :
            return self.LinkLeftArrow
        elif (linkArrow == LinkDirections.Right) :
            return self.LinkRightArrow
        elif (linkArrow == LinkDirections.BottomLeft) :
            return self.LinkLowerLeftArrow
        elif (linkArrow == LinkDirections.BottomRight) :
            return self.LinkLowerRightArrow
        elif (linkArrow == LinkDirections.TopLeft) :
            return self.LinkUpperLeftArrow
        elif (linkArrow == LinkDirections.TopRight) :
            return self.LinkUpperRightArrow
        else :
            print(f"ERROR : LayerNames.GetLinkArrowLayerNameFromEnum(linkArrow = {linkArrow.name}) : linkArrow not supported.")
            return None

In [157]:
class YugiohTranslation :
    
    def __init__(self, currentLanguage = "en") :
        self.currentLanguage = currentLanguage
        self.InitAttributeTranslations()
        self.InitMonsterArchetypeTranslations()
        self.InitMonsterRaceTranslations()
        self.InitAttributeTranslations()
        
    def InitAttributeTranslations(self) :
        self.spellTrapCardDesignation = dict()
        self.spellTrapCardDesignation["fr"] = {
            "Spell Card" : "Carte Magie",
            "Trap Card"  : "Carte Piège",
        }
        
    def InitAttributeTranslations(self) :
        self.attribute = dict()
        self.attribute["fr"] = {
            "DARK"   : "TÉNÈBRES",
            "DIVINE" : "DIVIN",
            "EARTH"  : "TERRE",
            "FIRE"   : "FEU",
            "LIGHT"  : "LUMIÈRE",
            "WATER"  : "EAU",
            "WIND"   : "VENT",
            "SPELL"  : "MAGIE",
            "TRAP"   : "PIÈGE"
        }
        
    def InitMonsterArchetypeTranslations(self) :
        self.monsterArchetype = dict()
        self.monsterArchetype["fr"] = {
            "Fusion"         : "Fusion",
            "Synchro"        : "Synchro",
            "XYZ"            : "XYZ",
            "Ritual"         : "Rituel",
            "Link"           : "Lien",
            "Pendulum"       : "Pendule",
            
            "Gemini"         : "Gémeau",
            "Spirit"         : "Spirit",
            "Toon"           : "Toon",
            "Union"          : "Union",
            "Flip"           : "Flip",
            "Tuner"          : "Synthoniseur",
            
            "Effect"         : "Effet",
            "Normal"         : "Normal"
        }
    
    def InitMonsterRaceTranslations(self) :
        self.monsterRace = dict()
        self.monsterRace["fr"] = {
            "Aqua"            : "Aqua",
            "Beast"           : "Bête",
            "Beast-Warrior"   : "Bête-Guerrier",
            "Creator-God"     : "Dieu Créateur",
            "Cyberse"         : "Cyberse",
            "Dinosaur"        : "Dinosaure",
            "Divine-Beast"    : "Bête Divine",
            "Dragon"          : "Dragon",
            "Fairy"           : "Elfe",
            "Fiend"           : "Démon",
            "Fish"            : "Poisson",
            "Insect"          : "Insecte",
            "Machine"         : "Machine",
            "Plant"           : "Plante",
            "Psychic"         : "Psychique",
            "Pyro"            : "Pyro",
            "Reptile"         : "Reptile",
            "Rock"            : "Rocher",
            "Serpent"         : "Serpent de Mer",
            "Spellcaster"     : "Magicien",
            "Thunder"         : "Tonnerre",
            "Warrior"         : "Guerrier",
            "Winged Beast"    : "Bête Ailée",
            "Wyrm"            : "Wyrm",
            "Zombie"          : "Zombie"
        }
    
    def GetTranslation(self, inputText, translationSource, typeForDebuggingPurpose, outputLanguage = None) :
        if (outputLanguage is None) :
            language = self.currentLanguage
            
        if (outputLanguage == "en") :
            return inputText
        
        if (outputLanguage not in translationSource.keys()) :
            print(f"ERROR : Translation -> Language ('{outputLanguage}') absent in {typeForDebuggingPurpose} translation.")
            return inputText
        
        if (inputText not in translationSource[outputLanguage].keys()) :
            print(f"ERROR : Translation -> '{inputText}' absent in {typeForDebuggingPurpose} translation for language '{outputLanguage}'.")
            return inputText
        
        return translationSource[outputLanguage][inputText]
    
    def TranslateAttribute(self, inputText, outputLanguage = None) :
        if (outputLanguage is None) :
            outputLanguage = self.currentLanguage
        return self.GetTranslation(inputText, self.attribute, "Attribute", outputLanguage)
    
    def TranslateMonsterArchetype(self, inputText, outputLanguage = None) :
        if (outputLanguage is None) :
            outputLanguage = self.currentLanguage
        return self.GetTranslation(inputText, self.monsterArchetype, "Monster Archetype", outputLanguage)
    
    def TranslateMonsterRace(self, inputText, outputLanguage = None) :
        if (outputLanguage is None) :
            outputLanguage = self.currentLanguage
        return self.GetTranslation(inputText, self.monsterRace, "Monster Race", outputLanguage)
    
    def TranslateSpellTrapCardDesignation(self, inputText, outputLanguage = None) :
        if (outputLanguage is None) :
            outputLanguage = self.currentLanguage
        return self.GetTranslation(inputText, self.spellTrapCardDesignation, "Spell & Trap Card Designation", outputLanguage)

In [158]:
class PhotoshopColorManager :
    def __init__(self) :
        self.Black  = self.GetColor((  0,   0,   0))
        self.White  = self.GetColor((255, 255, 255))
        self.Gold   = self.GetColor((219, 172,  52))
        self.Silver = self.GetColor((224, 224, 224))
        
    def GetColor(self, colorInRGB = None) :
        color = comtypes.client.CreateObject("Photoshop.SolidColor")
        color.RGB.Red   = colorInRGB[0]
        color.RGB.Green = colorInRGB[1]
        color.RGB.Blue  = colorInRGB[2]
        return color
    
    def GetColorFromEnum(self, enum = TextColor.Black) :
        if (enum == TextColor.Black) :
            return self.Black
        elif (enum == TextColor.White) :
            return self.White
        elif (enum == TextColor.Silver) :
            return self.Silver
        elif (enum == TextColor.Gold) :
            return self.Gold
        else :
            print(f"ERROR : PhotoshopColorManager.GetColorFromEnum({enum = }) -> Color not supported.")
            return None
            
PhotoshopColors = PhotoshopColorManager()

In [160]:
class YuGiOhCardPsd :
    def __init__(self,
                 path = r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\YuGiOhTemplate.psd",
                 printDebug = True,
                 printWarning = True,
                 printError = True,
                 setName = "CSTM",
                 setBeginNumber = 0,
                 edition = "ÉDITION LIMITÉE",
                 translationModule = None) :
        self.path                 = path
        self.psd                  = PSDImage.open(path)
        self.printDebug           = printDebug
        self.printWarning         = printWarning
        self.printError           = printError
        self.setName              = setName
        self.setNumber            = setBeginNumber
        self.textToChange         = dict()
        self.textColorToChange    = dict()
        self.cardType             = None
        self.PhotoshopApp         = comtypes.client.CreateObject("Photoshop.Application", dynamic = True)
        self.cardName             = None
        self.errors               = dict()
        self.errors["temp"]       = []
        
        self.invalidCharacters    = dict()
        self.GetInvalidCharacterReplacements()
        
        self.translationModule = translationModule
           
        self.InitPSD()
        
        
        if (edition is not None) :
            self.ChangeGroupOrLayerVisibility(LayerNames.Edition, True)
            self.ChangeTextOnLayer(LayerNames.Edition, edition)
        else :
            self.ChangeGroupOrLayerVisibility(LayerNames.Edition, False)
        
    def InitPSD(self) :
        # General
        self.ChangeGroupOrLayerVisibility("Bleed", True)
        self.ChangeGroupOrLayerVisibility("Copyright", True)
        self.ChangeGroupOrLayerVisibility(LayerNames.SerialNumber, True)
        """self.ChangeGroupOrLayerVisibility("$$$Card Name", True)
        for layer in self.GetGroupOrLayerByName("$$$Card Name") :
            layer.visible = False
        self.ChangeGroupOrLayerVisibility("$$$Card Name V1", True)
        self.ChangeGroupOrLayerVisibility("$$$Set ID", True)
        self.ChangeGroupOrLayerVisibility("$$$Serial Number", True)
        self.ChangeGroupOrLayerVisibility("$$$Gold Rare", False)
        self.ChangeGroupOrLayerVisibility("$$$Holo (Select 1)", False)
    
        # Monster
        self.ChangeGroupOrLayerVisibility("$$$Monster Frames", True)
        self.ChangeVisibilityForEveryLayerInsideGroup(LayerNames.AttributeGroup, False)
        self.ChangeGroupOrLayerVisibility(LayerNames.GetAttributeForLanguage(LayerNames.LocalisedAttributeGroup), True)
        self.ChangeGroupOrLayerVisibility("$$$Card Text", True)
        self.ChangeGroupOrLayerVisibility("$$$Levels", True)
        
        # S/T
        self.ChangeGroupOrLayerVisibility("$$$Spell/Trap Frames", True)
        self.ChangeGroupOrLayerVisibility("$$$Card Type/Icon", True)
        self.ChangeGroupOrLayerVisibility("$$$S/T Attribute", True)
        self.ChangeGroupOrLayerVisibility("$$$Effect Text", True)

        # Token
        self.ChangeGroupOrLayerVisibility("$$$Token Text", False)
        
        # All
        self.ChangeGroupOrLayerVisibility("$$$Levels", True)
        for layer in self.GetGroupOrLayerByName("$$$Levels") :
            layer.visible = False
        self.ChangeGroupOrLayerVisibility("$$$Current Levels", True)"""
        
### Prints --------------------------------------------------------------------------------------
        
    def PrintDebug (self, message) :
        if (self.printDebug):
            print("DEBUG : YuGiOhCardPsd." + message)
            
    def PrintWarning (self, message) :
        if (self.printWarning):
            print("WARNING : YuGiOhCardPsd." + message)
            
    def PrintError (self, message) :
        if (self.printError):
            print("ERROR : YuGiOhCardPsd." + message)
        self.errors["temp"].append(message)
    
    
    def PrintNotSupported(self, name, reason = ": see code :)") :
        print(f"NOT SUPPORTED : Returning None for function '{name}' because {reason}.")
    
        
    def SaveErrors(self) :
        if (len(self.errors["temp"]) > 0) :
            if (self.cardName in self.errors.keys()) :
                self.errors[self.cardName] += self.errors["temp"]
            else :
                self.errors[self.cardName] = self.errors["temp"]
            self.errors["temp"] = []
            
            return True
        else :
            return False
        
            
    def PrintHierarchy(self, isVisible = True, printOnlyVisibleLayers = False, visibilityCharacter = "X") :
        PrintLayersHierarchy(self.psd, 0, isVisible, printOnlyVisibleLayers, visibilityCharacter)
        
### Basic Functions --------------------------------------------------------------------------------------
        
    def GetGroupOrLayerByName(self, name, obj = None) :
        if (obj is None) :
            obj = self.psd
        
        if (obj.name == name) :
            return obj
        
        if (obj.is_group()) :
            for subObj in obj :
                result = self.GetGroupOrLayerByName(name, subObj)
                if (result != None) :
                    return result
        
        return None
        
    def GetGroupOrLayerPathFromRootByName(self, name, obj = None) :
        if (obj is None) :
            obj = self.psd
        
        if (obj.name == name) :
            return [obj.name]
        
        if (obj.is_group()) :
            for subObj in obj :
                result = self.GetGroupOrLayerPathFromRootByName(name, subObj)
                if (result != None) :
                    result.append(obj.name)
                    return result
        
        return None
    
    def IsLayerInBounds(self, textLayer, boundaries, limit = [0, 1, 2, 3]) :
        #print(f"textLayer in {textLayer.Bounds}\nboundaries in {boundaries}")
        for direction in limit :
            if (direction < Direction.Top) : #Left or Top
                if (textLayer.Bounds[direction] < boundaries[direction]) :
                     return False
            else :               #Right or Bottom
                if (textLayer.Bounds[direction] > boundaries[direction]) :
                     return False
        return True
    
    def ResizeText(self, textLayer, boundaries, limit = [0, 1, 2, 3], changeFontSize = True, psd_api = None, baseFontSize = 5) :
        minFontSize = 4.5
        fontIncrementInPt = 0.1
        scaleIncrementInPercent = 2
        self.PrintDebug(f"textLayer = '{textLayer.name}' \nboundaries = '{boundaries.name}' \n{limit = } \n{changeFontSize = }")
        saveAndReload = False
        if (psd_api is None) :
            saveAndReload = True
            self.psd.save(self.path)
            psd_api = self.PhotoshopApp.Open(self.path)
            
        # Mandatory because of an auto-conversion from pt to unit
        if (changeFontSize) :
            currentFontSize = baseFontSize
            textLayer.TextItem.Leading = currentFontSize # Auto-conversion
            textLayer.TextItem.Size = currentFontSize # Auto-conversion
        textLayer.TextItem.HorizontalScale = 100
        #textLayer.TextItem.VerticalScale = 100
        #iteration = 0
        if (not self.IsLayerInBounds(textLayer, boundaries.Bounds, limit)):
            while (not self.IsLayerInBounds(textLayer, boundaries.Bounds, limit)) :
                if (changeFontSize and currentFontSize > minFontSize):
                    currentFontSize -= fontIncrementInPt
                    textLayer.TextItem.Size = currentFontSize
                    textLayer.TextItem.Leading = currentFontSize
                elif (changeFontSize and currentFontSize <= minFontSize) :
                    textLayer.TextItem.Leading = currentFontSize * 0.95
                    
                textLayer.TextItem.HorizontalScale -= scaleIncrementInPercent
                #iteration += 1
                """print(f"### Iteration n°{iteration}")
                print(f"Size = {layer.TextItem.Size}")
                print(f"HorizontalScale = {layer.TextItem.HorizontalScale}")
                print(f"VerticalScale = {layer.TextItem.VerticalScale}")"""
            while (textLayer.TextItem.HorizontalScale < 100 and self.IsLayerInBounds(textLayer, boundaries.Bounds, limit)) :
                textLayer.TextItem.HorizontalScale += 1
            textLayer.TextItem.HorizontalScale -= 1
        else : 
            self.PrintDebug(f"ResizeText(textLayer = '{textLayer.name}' \n\tboundaries = '{boundaries.name}' \n\tlimit = '{limit}' \n\tchangeFontSize = '{changeFontSize}')" +
                           f"layer already in bounds.")
        
        if (saveAndReload) :
            psd_api.Save()
            psd_api.Close()
            self.psd = PSDImage.open(self.path)
    
    def ResizeAllTexts(self, psd_api = None) :
        saveAndReload = False
        if (psd_api is None) :
            saveAndReload = True
            self.psd.save(self.path)
            psd_api = self.PhotoshopApp.Open(self.path)
            
        for layerName in self.textToChange.keys():
            self.PrintDebug(f"ResizeAllTexts() -> Resizing text for layer '{layerName}'...")
            hierarchy = self.GetGroupOrLayerPathFromRootByName(layerName)
            layer = psd_api
            hierarchy.pop()
            while (len(hierarchy) > 0) :
                childName = hierarchy.pop()
                #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
                layer = layer.Layers[childName]
            
            if (layerName.lower().startswith("card name")) :
                boundsLayerName = LayerNames.CardNameBounds
                limit = [Direction.Right]
                changeFontSize = False
            else :
                boundsLayerName = f"{layerName} - Bounds"
                if (layerName.lower().startswith("set number")) :
                    limit = [Direction.Right]
                    changeFontSize = False
                else :
                    limit = [Direction.Bottom]#, Direction.Right]
                    changeFontSize = True
                
            boundsLayerFound = False
            for l in layer.parent.Layers :
                if (l.name == boundsLayerName) :
                    boundsLayerFound = True
                    #print(f"Bounds Layer Found => '{boundsLayerName}'")
                    break

            if (not boundsLayerFound) :
                self.PrintDebug(f"ResizeAllTexts() -> No Bounds Layer found for layer '{layerName}'.")
            else :
                boundaries = layer.parent.Layers[boundsLayerName]
                self.ResizeText(layer, boundaries, limit = limit, changeFontSize = changeFontSize, psd_api = psd_api)
        
        if (saveAndReload) :
            psd_api.Save()
            psd_api.Close()
            self.psd = PSDImage.open(self.path)
            
    def ChangeParentsVisibility(self, obj, visibility) :
        while (obj is not None and obj.name != "Root") :
            obj.visible = visibility
            obj = obj.parent
            
    
    def ChangeGroupOrLayerVisibility(self, name, visibility, makeParentVisibleIfVisibilityIsTrue = True) :
        
        obj = self.GetGroupOrLayerByName(name)
        if (obj != None) :
            if (obj.visible == visibility) :
                self.PrintDebug(f"ChangeGroupOrLayerVisibility({name = }) -> '{name}' is already {'visible' if visibility else 'hidden'}.")
            else :
                obj.visible = visibility
                
            if (visibility and makeParentVisibleIfVisibilityIsTrue) :
                self.ChangeParentsVisibility(obj.parent, True)
        else :
            self.PrintError(f"ChangeGroupOrLayerVisibility({name = }) -> Cannot find a group or layer named '{name}'.")
    
    def ChangeVisibilityForEveryLayerInsideGroup(self, groupName, visibility, recursive = False) :
        for layer in self.GetGroupOrLayerByName(groupName) :
            layer.visible = visibility
            if (recursive and layer.is_group()) :
                for child in layer :
                    if (child.is_group()) :
                        self.ChangeVisibilityForEveryLayerInsideGroup(child.name, visibility, recursive)
                    else :
                        child.visible = visibility
            
    def MakeNormalMonsterTextItalics(self, psd_api) :
        if ((LayerNames.NormalMonsterEffect.strip() == "") or 
            (LayerNames.MonsterEffect != LayerNames.NormalMonsterEffect)) :
            return
        
        hierarchy = self.GetGroupOrLayerPathFromRootByName(LayerNames.MonsterEffect)
        layer = psd_api
        hierarchy.pop()
        while (len(hierarchy) > 0) :
            childName = hierarchy.pop()
            #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
            layer = layer.Layers[childName]
        if (self.cardType == CardType.MonsterNormal) :
            layer.TextItem.FauxItalic = True
        else :
            layer.TextItem.FauxItalic = False
        
         
    def ChangeAllTextValues(self, psd_api = None) :
        saveAndReload = False
        if (psd_api is None) :
            saveAndReload = True
            self.psd.save(self.path)
            psd_api = self.PhotoshopApp.Open(self.path)
            
        for layerName in self.textToChange.keys():
            try :
                text = self.textToChange[layerName]
                self.PrintDebug(f"ChangeAllTextValues() -> Setting '{text}' to layer '{layerName}'...")
                hierarchy = self.GetGroupOrLayerPathFromRootByName(layerName)
                layer = psd_api
                hierarchy.pop()
                while (len(hierarchy) > 0) :
                    childName = hierarchy.pop()
                    #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
                    layer = layer.Layers[childName]
                layer.TextItem.Contents = text   
            except:
                self.PrintError(f"ChangeAllTextValues() -> Error in setting '{text}' to layer '{layerName}' : {sys.exc_info()[0]}")

        
        if (saveAndReload) :
            psd_api.Save()
            psd_api.Close()
            self.psd = PSDImage.open(self.path)
            
    def ChangeAllTextColors(self, psd_api = None) :
        saveAndReload = False
        if (psd_api is None) :
            saveAndReload = True
            self.psd.save(self.path)
            psd_api = self.PhotoshopApp.Open(self.path)
            
        for layerName in self.textColorToChange.keys():
            try :
                color = self.textColorToChange[layerName]
                if (color == PhotoshopColors.White) :
                    colorName = "'White'"
                elif (color == PhotoshopColors.Black) :
                    colorName = "'Black'"
                else :
                    colorName = "'Other'"
                self.PrintDebug(f"ChangeAllTextColors() -> Setting color {colorName} to layer '{layerName}'...")
                hierarchy = self.GetGroupOrLayerPathFromRootByName(layerName)
                layer = psd_api
                hierarchy.pop()
                while (len(hierarchy) > 0) :
                    childName = hierarchy.pop()
                    #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
                    layer = layer.Layers[childName]
                layer.TextItem.Color = color   
            except:
                self.PrintError(f"ChangeAllTextColors() -> Error in setting color {colorName} to layer '{layerName}' : {sys.exc_info()[0]}")

        
        if (saveAndReload) :
            psd_api.Save()
            psd_api.Close()
            self.psd = PSDImage.open(self.path)
            
    def ChangeAllTexts(self, psd_api = None) :
        saveAndReload = False
        if (psd_api is None) :
            saveAndReload = True
            self.psd.save(self.path)
            psd_api = self.PhotoshopApp.Open(self.path)
            
        self.ChangeAllTextColors(psd_api)
        self.ChangeAllTextValues(psd_api)
        self.ResizeAllTexts(psd_api)
        
        self.textToChange.clear()
        self.textColorToChange.clear()
        
        if (saveAndReload) :
            psd_api.Save()
            psd_api.Close()
            self.psd = PSDImage.open(self.path)
            
    def SelectFromBoundaries(self, boundaries, app = None) :
        if (app is None) :
            app = self.PhotoshopApp
        app.ActiveDocument.Selection.Select(((boundaries[0], boundaries[1]), 
                                            (boundaries[0], boundaries[3]), 
                                            (boundaries[2], boundaries[3]), 
                                            (boundaries[2], boundaries[1])))
    
    def SetPicture(self, picturePath, psd_api = None, pendulumSize = None) :
        saveAndReload = False
        if (psd_api is None) :
            saveAndReload = True
            self.psd.save(self.path)
            psd_api = self.PhotoshopApp.Open(self.path)
            
        if (pendulumSize is not None) :
            pendulumSizeAsString = f"[{pendulumSize}] "
        else :
            pendulumSizeAsString = ""          
        
        hierarchy = self.GetGroupOrLayerPathFromRootByName(pendulumSizeAsString + LayerNames.SampleImage)
        layer = psd_api
        hierarchy.pop()
        while (len(hierarchy) > 0) :
            childName = hierarchy.pop()
            #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
            layer = layer.Layers[childName]
        boundaries = layer.bounds
        group = layer.parent
        try :
            group.ArtLayers.Remove(group.Layers[pendulumSizeAsString + LayerNames.CurrentArtwork])
        except :
            self.PrintWarning(f"SetPicture(...) : no layer named '{pendulumSizeAsString + LayerNames.CurrentArtwork}'")
        self.PhotoshopApp.Load(picturePath)
        self.PhotoshopApp.ActiveDocument.ResizeImage(boundaries[2] - boundaries[0], boundaries[3] - boundaries[1])

        self.PhotoshopApp.ActiveDocument.Selection.SelectAll()
        self.PhotoshopApp.ActiveDocument.Selection.Copy()
        self.PhotoshopApp.ActiveDocument.Close(2)
        self.SelectFromBoundaries(boundaries)
        self.PhotoshopApp.ActiveDocument.ActiveLayer = layer
        self.PhotoshopApp.ActiveDocument.Paste()#IntoSelection()
        self.PhotoshopApp.ActiveDocument.ActiveLayer.Name = pendulumSizeAsString + LayerNames.CurrentArtwork
        
        if (saveAndReload) :
            psd_api.Save()
            psd_api.Close()

            self.psd = PSDImage.open(self.path)

    def ChangeTextOnLayer(self, layerName, text, makeVisible = True) :
        limitForLoggingPurpose = 25
        if (len(text) > limitForLoggingPurpose) :
            textForLoggingPurpose = f"{text[:limitForLoggingPurpose]}[...]"
        else :
            textForLoggingPurpose = text
        if (self.GetGroupOrLayerPathFromRootByName(layerName) is None) :
            self.PrintError(f"ChangeTextOnLayer({layerName = }, text = '{textForLoggingPurpose}') -> Cannot find a layer named '{layerName}'.")
            return False
        if (layerName in self.textToChange.keys()) :
            self.PrintWarning(f"ChangeTextOnLayer({layerName = }, text = '{textForLoggingPurpose}') -> textToChange already contains " + 
                              f"a value for {layerName} : '{self.textToChange[layerName]}'.")
        else :
            self.PrintDebug(f"ChangeTextOnLayer({layerName = }, text = '{textForLoggingPurpose}') -> setting " + 
                              f"value for {layerName} => '{textForLoggingPurpose}'.")
        self.textToChange[layerName] = text
        if (makeVisible) :
            self.ChangeGroupOrLayerVisibility(layerName, True)
        return True
        
    def GetInvalidCharacterReplacements(self) :
        self.invalidCharacters["●"] = "● "
        self.invalidCharacters["\n"] = "\r"
        
    def ReplaceInvalidCharacters(self, text, extraInvalidToDelete = []) :
        tempInvalidCharacters = self.invalidCharacters.copy()
        for extraChar in extraInvalidToDelete :
            tempInvalidCharacters[extraChar] = ""
        for character in tempInvalidCharacters.keys() :
            text = text.replace(character, tempInvalidCharacters[character])
        return text
    
    def ChangeTextLayerColor(self, layerName, textColorAsEnum = None, textColorAsRGB = (0, 0, 0)) :
        if (textColorAsEnum is not None) :
            color = PhotoshopColors.GetColorFromEnum(textColorAsEnum)
            if (color is not None) :
                self.textColorToChange[layerName] = color
                return True
            
        if (textColorAsRGB is not None) : 
            color = PhotoshopColors.GetColor(textColorAsRGB)
            if (color is not None) :
                self.textColorToChange[layerName] = color
                return True
                
        self.PrintError(f"ChangeTextLayerColor({textColorAsEnum = }, {textColorAsRGB}) -> Cannot get a color for these parameters.")
        return False
        
### Practical Functions --------------------------------------------------------------------------------------
    
    def MakeCardType(self, cardType) :
        """if (cardType is None) :
            if (self.cardType != None) :
                cardType = self.cardType
            else :
                self.PrintError(f"MakeCardType(...) -> cardType and self.cardType are both 'None'.")
                return
        else :
            self.cardType = cardType"""
        
        self.cardType = cardType
            
        
        self.ChangeVisibilityForEveryLayerInsideGroup(LayerNames.FrameGroup, False)
        self.ChangeGroupOrLayerVisibility(LayerNames.MonsterGroup, False)
        self.ChangeGroupOrLayerVisibility(LayerNames.SpellTrapGroup, False)
        self.ChangeGroupOrLayerVisibility(LayerNames.STTypesGroup, False)
        
        cardFrameLayerName = LayerNames.GetFrameLayerNameFromCardType(cardType)
        self.ChangeGroupOrLayerVisibility(cardFrameLayerName, True)
        
        self.ChangeGroupOrLayerVisibility(LayerNames.LinkTools, cardType == CardType.MonsterLink)
        self.ChangeGroupOrLayerVisibility(LayerNames.PendulumTools, False)
        
        
#        for layerName in LayerNames.GetAllTextInFrameFromCardType(cardType) :
#            if (cardType == CardType.MonsterXyz) :
#                self.ChangeTextLayerColor(layerName, textColorAsEnum = TextColor.White)
#            else :
#                self.ChangeTextLayerColor(layerName, textColorAsEnum = TextColor.Black)
        
        cardArchetype = cardType // 10
        
        textOnFrameShouldBeWhite = (cardType == CardType.MonsterXyz) #or (cardArchetype == CardType.Spell or cardArchetype == CardType.Trap)
        self.ChangeGroupOrLayerVisibility(LayerNames.LayerThatMakesTextOnFrameWhite, textOnFrameShouldBeWhite)
        
        if (cardArchetype == CardType.Spell or cardArchetype == CardType.Trap) :
            self.ChangeVisibilityForEveryLayerInsideGroup(LayerNames.STTypesGroup, False)
            self.ChangeGroupOrLayerVisibility(LayerNames.NormalSpellCard, False)
            self.ChangeGroupOrLayerVisibility(LayerNames.NormalTrapCard, False)
            self.ChangeGroupOrLayerVisibility(LayerNames.SpecialSpellCard, False)
            self.ChangeGroupOrLayerVisibility(LayerNames.SpecialTrapCard, False)
            stType = LayerNames.GetSpellTrapTypeFromCardType(cardType)
            if (stType == None) :
                if (cardArchetype == CardType.Spell) : 
                    self.ChangeGroupOrLayerVisibility(LayerNames.NormalSpellCard, True)
                elif (cardArchetype == CardType.Trap) : 
                    self.ChangeGroupOrLayerVisibility(LayerNames.NormalTrapCard, True)
                else :
                    self.PrintError(f"MakeCardType(cardType = {cardType.name}) -> You're not supposed to get here... (1)")
            else :
                if (cardArchetype == CardType.Spell) : 
                    self.ChangeGroupOrLayerVisibility(LayerNames.SpecialSpellCard, True)
                elif (cardArchetype == CardType.Trap) : 
                    self.ChangeGroupOrLayerVisibility(LayerNames.SpecialTrapCard, True)
                else :
                    self.PrintError(f"MakeCardType(cardType = {cardType.name}) -> You're not supposed to get here... (2)")
                self.ChangeGroupOrLayerVisibility(stType, True)
                
                 
    def SetCardText(self, text, cardType = None) :
        if (cardType is None) :
            if (self.cardType != None) :
                cardType = self.cardType
            else :
                self.PrintError(f"SetCardText(...) -> cardType and self.cardType are both 'None'.")
                return
            
        cardArchetype = cardType // 10
        
        textLayerName = LayerNames.GetCardEffetBoxFromCardType(cardType)
        
        if (textLayerName is None) :
            self.PrintError(f"ChangeTextOnLayer('{text = }', {cardType = }) -> Card type not managed.")
            return False
        
        for layerName in [LayerNames.MonsterEffect,
                         LayerNames.NormalMonsterEffect,
                         LayerNames.SpellTrapEffect,
                         LayerNames.TokenMonsterFrame] :
            self.ChangeGroupOrLayerVisibility(layerName, False)
        
        #self.ChangeGroupOrLayerVisibility(textLayerName, True) # Useless --------------------v 
        self.ChangeTextOnLayer(textLayerName, self.ReplaceInvalidCharacters(text))# makeVisible = True (implicit)
    
    def SetCardName(self, cardName, nameColor = TextColor.Black, cardType = None) :
        if (cardType is None) :
            if (self.cardType != None) :
                cardType = self.cardType
            else :
                self.PrintError(f"SetCardName(...) -> cardType and self.cardType are both 'None'.")
                return
            
        self.cardName = cardName
        
        if (nameColor == TextColor.Black and 
            ((self.cardType in [CardType.MonsterXyz, CardType.MonsterLink]) or 
             ((cardType // 10) in [CardType.Spell, CardType.Trap]))) :
            nameColor = TextColor.White
        
        layerName = LayerNames.GetCardNameLayerNameFromColor(nameColor)
        
        self.ChangeVisibilityForEveryLayerInsideGroup(LayerNames.CardNamesGroup, False)
        # Implicit Here --v  self.ChangeGroupOrLayerVisibility(layerName, True)
        self.ChangeTextOnLayer(layerName, self.ReplaceInvalidCharacters(cardName,['\n', '\r']).strip())
    
    def SetMonsterType(self, text) :
        self.ChangeTextOnLayer(LayerNames.MonsterRace, text)
        
    def GetValueAsFormattedString(self, value, numberOfChar, leadingValue = "  ") :
        if (type(value) == int) :
            result = f"{value:>{numberOfChar}}"
        elif (type(value) == str) :
            result = value.strip().lstrip('0')
            if (result == "") :
                result = "0"
            result = f"{result:>{numberOfChar}}"

        result = result.replace(" ", leadingValue)
        return(result)
    
    def SetAtkAndLinkValue(self, atkValue, linkValue) :
        # "?" Layer only contains the value (not "ATK/")
        #self.ChangeVisibilityForEveryLayerInsideGroup(LayerNames.AtkDefGroup, False)
        atkString = "/" # Label is on another layer : ATKLabel # Old : "ATK/"
        if (atkValue == "?") :
            self.ChangeGroupOrLayerVisibility(LayerNames.ATKisQuestionMark, True)
        else :
            self.ChangeGroupOrLayerVisibility(LayerNames.ATKisQuestionMark, False)
            #self.ChangeGroupOrLayerVisibility(LayerNames.ATK, True)
            atkString += self.GetValueAsFormattedString(atkValue, 4, "  ")
        self.ChangeTextOnLayer(LayerNames.ATK, atkString)
            
        
        self.ChangeGroupOrLayerVisibility(LayerNames.DEFisQuestionMark, False)
        self.ChangeGroupOrLayerVisibility(LayerNames.DEF, False)
        self.ChangeTextOnLayer(LayerNames.LinkNumber, linkValue)
    
    def SetAtkAndDef(self, atkValue, defValue) :
        # "?" Layer only contains the value (not "ATK/")
        #self.ChangeVisibilityForEveryLayerInsideGroup(LayerNames.AtkDefGroup, False)
        atkString = "/" # Label is on another layer : ATKLabel # Old : "ATK/"
        if (atkValue == "?") :
            self.ChangeGroupOrLayerVisibility(LayerNames.ATKisQuestionMark, True)
        else :
            self.ChangeGroupOrLayerVisibility(LayerNames.ATKisQuestionMark, False)
            #self.ChangeGroupOrLayerVisibility(LayerNames.ATK, True)
            atkString += self.GetValueAsFormattedString(atkValue, 4, "  ")
        self.ChangeTextOnLayer(LayerNames.ATK, atkString)
            
        
        defString = "DEF/"
        if (defValue == "?") :
            self.ChangeGroupOrLayerVisibility(LayerNames.DEFisQuestionMark, True)
        else :
            self.ChangeGroupOrLayerVisibility(LayerNames.DEFisQuestionMark, False)
            #self.ChangeGroupOrLayerVisibility(LayerNames.DEF, True)
            defString += self.GetValueAsFormattedString(defValue, 4, "  ")
        self.ChangeTextOnLayer(LayerNames.DEF, defString)
    
    def SetCollectorNumber(self, number = None, cardType = None, layerName = None) :
        if (cardType is None) :
            if (self.cardType != None) :
                cardType = self.cardType
            else :
                self.PrintError(f"SetCollectorNumber(...) -> cardType and self.cardType are both 'None'.")
                return
            
        if (number is None) :
            number = self.setNumber 
            self.setNumber += 1
        collectorText = f"{self.setName}-FR{self.setNumber:03d}"
        
        self.ChangeGroupOrLayerVisibility(LayerNames.SetNumberNormal, False)
        self.ChangeGroupOrLayerVisibility(LayerNames.SetNumberLink, False)
        self.ChangeGroupOrLayerVisibility(LayerNames.SetNumberPendulum, False)
        
        if (layerName == None) :
            layerName = LayerNames.GetSetNumberLayerNameFromCardType(cardType)
        self.ChangeTextOnLayer(layerName, collectorText)
        return number
    
    def SetMonsterTypeRightBracket(self) :
        self.PrintNotSupported(SetMonsterTypeRightBracket)
        return
        ## Useless
        typeLayer = self.GetGroupOrLayerByName("$$$Type/Subtype")
        bracketLayer = self.GetGroupOrLayerByName("$$$]")
        bracketLayer.left = typeLayer.right + 4
        
    def SetAttribute(self, attribute, language = "en") :
        #self.ChangeVisibilityForEveryLayerInsideGroup(LayerNames.GetAttributeForLanguage(LayerNames.LocalisedAttributeGroup, language), False)
        self.ChangeVisibilityForEveryLayerInsideGroup(LayerNames.AttributeGroup, visibility = False, recursive = True)
        self.ChangeGroupOrLayerVisibility(LayerNames.GetAttributeForLanguage(attribute.upper(), language), True)
    
    def SetLevelOrRank(self, level, isRank = False, psd_api = None) :
        saveAndReload = False
        if (psd_api is None) :
            saveAndReload = True
            self.psd.save(self.path)
            psd_api = self.PhotoshopApp.Open(self.path)
            
        if (level is None) :
            level = 0
            
        hierarchy = self.GetGroupOrLayerPathFromRootByName(LayerNames.LevelRankGroup)
        levelGroup = psd_api
        hierarchy.pop()
        while (len(hierarchy) > 0) :
            childName = hierarchy.pop()
            #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
            levelGroup = levelGroup.Layers[childName]
            
        try :
            levelGroup.ArtLayers.Remove(levelGroup.Layers["Current Levels"])
        except :
            self.PrintWarning("SetLevelOrRank(...) : no layer named 'Current Levels'")
                
                
        if (not isRank) :
            layer = levelGroup.Layers[LayerNames.Levels]
        else :
            layer = levelGroup.Layers[LayerNames.Ranks]
        
        """if (level < 12) :
            if (not isRank) :
                layer = levelGroup.Layers["Levels - 11"]
            else :
                layer = levelGroup.Layers["Ranks - 11"]
        elif (level == 12) :
            if (not isRank) :
                layer = levelGroup.Layers["Levels - 12"]
            else :
                layer = levelGroup.Layers["Ranks - 12"]
        else :
            self.PrintError(f"SetLevelOrRank(level = {level}, isRank = {isRank}) : number not supported.")"""
        
        self.PhotoshopApp.ActiveDocument.ActiveLayer = layer.Duplicate()
        
        self.PhotoshopApp.ActiveDocument.ActiveLayer.Name = "Current Levels"
        maxLevel = 12
        
        shift = 0
        """if (not isRank and level >= 6) :
            shift = -1
        elif (isRank and level >= 9) :
            shift = 1
        elif (isRank and level <= 2) :
            shift = -1"""
        
        if (level < 12) :
            boundaries = self.PhotoshopApp.ActiveDocument.ActiveLayer.bounds
            if (level == 0) :
                newBoundaries = boundaries
            elif (not isRank) :
                whereToCut = boundaries[0] + ((boundaries[2] - boundaries[0]) * (1 - level / maxLevel)) + shift
                newBoundaries = (boundaries[0], boundaries[1], whereToCut, boundaries[3])
            else :
                whereToCut = boundaries[2] - ((boundaries[2] - boundaries[0]) * (1 - level / maxLevel)) + shift
                newBoundaries = (whereToCut, boundaries[1], boundaries[2], boundaries[3])
            self.SelectFromBoundaries(newBoundaries)
            self.PhotoshopApp.ActiveDocument.Selection.Clear()
        
        if (saveAndReload) :
            psd_api.Save()
            psd_api.Close()

            self.psd = PSDImage.open(self.path)
        
    def SetSerialNumber(self, text) :
        if (type(text) == int) :
            text = f"{text}"
        
        self.ChangeTextOnLayer(LayerNames.SerialNumber, text)    
        
    def SetLinkArrowsVisibility(self, activeArrows) :
        layerNames = []
        for aa in activeArrows :
            layerNames.append(LayerNames.GetLinkArrowLayerNameFromEnum(aa))
        arrowsGroup = self.GetGroupOrLayerByName(LayerNames.LinkArrowsGroup)
        for arrow in arrowsGroup :
            isActive = arrow.name in layerNames
            for activeOrNot in arrow :
                activeOrNot.visible = ((activeOrNot.name == LayerNames.LinkArrowActive) == isActive)
                
                
### Pendulum -------------------------------------------------------------------------------------------------------
    def GetPendulumSize(self) :
        size = "M"
        
        if (size != "S") :
            self.ChangeGroupOrLayerVisibility(LayerNames.PendulumSmallGroup, False)
        if (size != "M") :
            self.ChangeGroupOrLayerVisibility(LayerNames.PendulumMediumGroup, False)
        if (size != "L") :
            self.ChangeGroupOrLayerVisibility(LayerNames.PendulumLargeGroup, False)
            
        self.ChangeGroupOrLayerVisibility(self.GetPendulumGroupFromSize(size), True)
        
        return size
        
    def ChangePendulumVisibility(self, visibility) :
        self.ChangeGroupOrLayerVisibility(LayerNames.PendulumTools, visibility)
        self.ChangeGroupOrLayerVisibility(LayerNames.PendulumThingsToHide, not visibility)
    
    def GetPendulumGroupFromSize(self, size = "M") :
        if (size == "S") :
            return LayerNames.PendulumSmallGroup
        elif (size == "M") :
            return LayerNames.PendulumMediumGroup
        elif (size == "L") :
            return LayerNames.PendulumLargeGroup
        else :
            self.PrintError(f"GetPendulumGroupFromSize({size = }) -> Size not supported")
        
    
    def SetPendulumCardText(self, monsterText, spellText, size = "M", cardType = None) :
        if (cardType is None) :
            if (self.cardType != None) :
                cardType = self.cardType
            else :
                self.PrintError(f"SetPendulumCardText(...) -> cardType and self.cardType are both 'None'.")
                return
            
        effectMonsterText = f"[{size}] " + LayerNames.PendulumMonsterEffect
        normalMonsterText = f"[{size}] " + LayerNames.PendulumNormalMonsterEffect
        if (cardType == CardType.MonsterNormal) :
            monsterTextLayerName = normalMonsterText
            self.ChangeGroupOrLayerVisibility(effectMonsterText, False) 
        else :
            monsterTextLayerName = effectMonsterText
            self.ChangeGroupOrLayerVisibility(normalMonsterText, False)
        self.ChangeTextOnLayer(monsterTextLayerName, monsterText)
        self.ChangeTextOnLayer(f"[{size}] " + LayerNames.PendulumSpellEffect, spellText) 
    
    def SetPendulumScales(self, scale, size = "M") :
        self.ChangeTextOnLayer(f"[{size}] " + LayerNames.PendulumScaleRed, f"{scale}")
        self.ChangeTextOnLayer(f"[{size}] " + LayerNames.PendulumScaleBlue, f"{scale}")
    
    def SetPendulumMonsterRace(self, monsterRace, size = "M") :
        self.ChangeTextOnLayer(f"[{size}] " + LayerNames.PendulumMonsterRace, monsterRace)
        
        
### Finalize -------------------------------------------------------------------------------------------------------
        
    def ChangeTextsAndPicture(self, picturePath, pendulumSize = None) :
        self.psd.save(self.path)
        
        psd_api = self.PhotoshopApp.Open(self.path)
        
        self.ChangeAllTexts(psd_api)
        """
        psd_api.Save()
        psd_api.Close()
        time.sleep(1)
        psd_api = self.PhotoshopApp.Open(self.path)
        """
        self.SetPicture(picturePath, psd_api, pendulumSize)
        
        psd_api.Save()
        psd_api.Close()
        """
        self.ChangeAllTexts()
        self.SetPicture(picturePath)
        """

        self.psd = PSDImage.open(self.path)
        
    def ChangeTextsAndPictureAndLevel(self, picturePath, level, isRank = False, pendulumSize = None) :
        self.psd.save(self.path)
        
        psd_api = self.PhotoshopApp.Open(self.path)
        
        self.ChangeAllTexts(psd_api)
        
        """
        psd_api.Save()
        psd_api.Close()
        time.sleep(1)
        psd_api = self.PhotoshopApp.Open(self.path)
        """
        self.SetPicture(picturePath, psd_api, pendulumSize)
        
        """ 
        psd_api.Save()
        psd_api.Close()
        time.sleep(1)
        psd_api = self.PhotoshopApp.Open(self.path)
        """
        
        self.SetLevelOrRank(level, isRank, psd_api)
        
        psd_api.Save()
        psd_api.Close()
        """
        self.ChangeAllTexts()
        self.SetPicture(picturePath)
        self.SetLevelOrRank(level, isRank)
        """

        self.psd = PSDImage.open(self.path)
    
    def SavePsd(self, directoryPath = None, filename = None) :
        if (directoryPath is None) :
            index = self.path.rindex('\\') + 1
            directoryPath = f"{self.path[0:index]}PSDs"
        if (filename is None) :
            if (self.cardName != None) :
                valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
                filename = ''.join(c for c in self.cardName if c in valid_chars)
                filename = filename.replace(' ','_') # I don't like spaces in filenames.
        fullPath = f"{directoryPath}\\{filename}.psd"
        self.psd.save(fullPath)
        
        self.SaveErrors()
        
        return fullPath
    
    def RenderPng(self, directoryPath = None, filename = None) :
        self.psd.save(self.path)
        psd_api = win32com.client.Dispatch("Photoshop.Application").Open(self.path)
            
        if (directoryPath is None) :
            index = self.path.rindex('\\') + 1
            directoryPath = f"{self.path[0:index]}Cards"
        if (filename is None) :
            if (self.cardName != None) :
                valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
                filename = ''.join(c for c in self.cardName if c in valid_chars)
                filename = filename.replace(' ','_') # I don't like spaces in filenames.
        fullPath = f"{directoryPath}\\{filename}.png"
        
        options = win32com.client.Dispatch("Photoshop.ExportOptionsSaveForWeb")
        options.Format = 13   # PNG Format
        options.PNG8 = False  # Sets it to PNG-24 bit
        options.Quality = 100

        psd_api.Export(ExportIn=fullPath, ExportAs=2, Options=options)
        
        #self.psd.composite(force=True).save(fullPath)
        
        self.SaveErrors()
        
        psd_api.Save()
        psd_api.Close()
        self.psd = PSDImage.open(self.path)
        
        return fullPath

In [161]:
class PrintProgression :
    def __init__(self) :
        self.printProgressionParamaters = dict()
        
    def initPrintProgression (self, name, startTime, totalItems, printEveryNthItem = 100, printName = False, overwrite = False) :
        if (not overwrite and name in self.printProgressionParamaters.keys()) :
            print(f"initPrintProgression (name = '{name}', overwrite = '{overwrite}', ...)" + 
                  f" : already have an entry for name '{name}' in printProgressionParamaters and overwrite is False -> Skipping")
            return self.printProgressionParamaters[name]
        parameters = dict()
        parameters["name"] = name
        parameters["startTime"] = startTime
        parameters["totalItems"] = totalItems
        parameters["printName"] = printName
        parameters["printEveryNthItem"] = printEveryNthItem
        self.printProgressionParamaters[name] = parameters
        return parameters
        
    def printProgression(self, name, itemCount = 0) :
        if (name not in self.printProgressionParamaters.keys()) :
            print(f"printProgression (name = '{name}', itemCount = '{itemCount}')" + 
                  f" : no entry for name '{name}' in printProgressionParamaters -> use initPrintProgression before")
            return
        
        parameters = self.printProgressionParamaters[name]
        startTime = parameters["startTime"]
        totalItems = parameters["totalItems"]
        printName = parameters["printName"]
        printEveryNthItem = parameters["printEveryNthItem"]
        
        if itemCount > 0 and itemCount % printEveryNthItem == 0 :
            
            display.clear_output(wait=True)
            totalItemsString = "{:,}".format(totalItems).replace(",", " ")
            countItemsString = "{:,}".format(itemCount).replace(",", " ")
            percentageString = "{:.4}%".format(100.0 * itemCount / totalItems)
            timeElapsed = time.time() - startTime
            eta = int((totalItems - itemCount) * timeElapsed / itemCount)
            namePrinted = ""
            if (printName) :
                namePrinted = f"{name} ->"
            string = (f"{namePrinted} {countItemsString} / {totalItemsString} done ({percentageString})" + 
                f" > {hmsString(timeElapsed)} : {hmsString(eta)} ({hmsString(timeElapsed + eta)})")
            print(string)


In [162]:
class YuGiOhCardInfos :
    def __init__(self,
                 language = "fr",
                 picklePath = r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\Database\CardInfos.pkl",
                 artworksDirectoryPath = r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\Artworks",
                 initArtworkSelector = True) :
        self.language = language
        self.picklePath = picklePath
        try :
            self.df = pd.read_pickle(picklePath)
        except FileNotFoundError :
            self.df = pd.DataFrame(columns = ["id", "name", "card_textdesc", "race", "cardTypeEnum", #"frameType", 
                                              "type", "name_en", "atk", "def", "level", 
                                              "attribute", "artworksUrls", "linkval", "linkmarkers", 
                                              "pendulum_effect", "pendulum_scale"])
            self.df.set_index("name", inplace = True)
        self.printProgression = PrintProgression() 
        self.errors = []
        self.aliases = dict()
        self.artworksDirectoryPath = artworksDirectoryPath
        if (initArtworkSelector) :
            self.InitArtworkSelector()
        self.cardRepetitions = dict()
        
    def PrintError (self, errorText, inputText = None, specificClassName = "YuGiOhCardInfos") :
        if (inputText is None) :
            inputText = ""
        else :
            self.errors.append((inputText, errorText))
            inputText = f"(for input = '{inputText}')"
        print(f"ERROR : {specificClassName}.{errorText}{inputText}")
        
    def GetLinkArrowsAsAListOfEnum(self, inputValue) :
        result = []
        
        if ("Top" in inputValue) :
            result.append(LinkDirections.Top)
        if ("Bottom" in inputValue) :
            result.append(LinkDirections.Bottom)
        if ("Left" in inputValue) :
            result.append(LinkDirections.Left)
        if ("Right" in inputValue) :
            result.append(LinkDirections.Right)
        if ("Bottom-Left" in inputValue) :
            result.append(LinkDirections.BottomLeft)
        if ("Bottom-Right" in inputValue) :
            result.append(LinkDirections.BottomRight)
        if ("Top-Left" in inputValue) :
            result.append(LinkDirections.TopLeft)
        if ("Top-Right" in inputValue) :
            result.append(LinkDirections.TopRight)
            
        return result
    
    def ConvertAllFields(self, cardInfo) :
        keys = list(cardInfo.keys())
        for fieldName in keys :
            if ((fieldName in self.fieldCorrespondance.keys()) and (fieldName != self.fieldCorrespondance[fieldName])) :
                newFieldName = self.fieldCorrespondance[fieldName]
                if ((newFieldName in cardInfo.keys()) and (cardInfo[newFieldName] != cardInfo[fieldName])) :
                    print(f"WARNING : ConvertAllFields(...) -> '{newFieldName}' already in cardInfo and {cardInfo[fieldName] = } != {cardInfo[newFieldName]} -> Overwriting...")
                cardInfo[newFieldName] = cardInfo[fieldName]
                cardInfo.pop(fieldName)
        return cardInfo
        
    def GetCardInfoByName(self, name) :
        raise NotImplementedError
    
    def SavePickle(self, overwrite = True, path = None) :
        if (path is None) :
            path = self.picklePath
        self.df.to_pickle(path)
   
    
### Artworks

    def GetArtworkPath(self, cardName, repetition = None, fromTemp = False, returnOrder = False) :
        if (not ("artworks" in self.df.columns)):# or not cardInDf) :
            cardInfo = self.df.loc[cardName]
            print("WARNING : GetArtworkPath(...) -> Artwork selection disabled, run SelectArtworkForAllDf to enable it.")
            result = self.GetCardArtworkPathSelectionDisabled(cardInfo["name"], cardInfo["artworksUrls"])
            if (returnOrder) :
                return (0, result)
            else :
                return result
            #This...
            
        availableArtworks = self.GetArtworksOrder(cardName, fromTemp)
         
        cardInDf = (type(availableArtworks) != dict()) and (availableArtworks is None)
        
        #...was Here
        
        if (repetition is None) : 
            if (not (cardName in self.cardRepetitions.keys())) :
                #print("Not Found")
                self.cardRepetitions[cardName] = 1
                repetition = 1
            else :
                self.cardRepetitions[cardName] += 1
                repetition = self.cardRepetitions[cardName] 
                #print(f"Setting self.cardRepetitions[cardName] to {self.cardRepetitions[cardName]}")
        else :
            #print("Forced")
            self.cardRepetitions[cardName] = repetition
            
        if (repetition == 1) :
            orderId = 1
        else :
            printableArtworksOrderId = list(filter(lambda k : k > 0, availableArtworks))
            nRep = repetition
            if (nRep <= len(printableArtworksOrderId)) :
                orderId = nRep
            else :
                i = 0
                cycles = 0
                artworksToRemove = list()
                while (nRep > 1) :
                    orderId = printableArtworksOrderId[i]
                    artwork = availableArtworks[orderId]
                    
                    if ((artwork["printLimit"] != 0) and (artwork["printLimit"] >= cycles)) :
                        artworksToRemove.append(orderId)
                    
                    if (i >= len(printableArtworksOrderId) - 1) :
                        for indexx in artworksToRemove :
                            #print(f"Removing {indexx}...")
                            printableArtworksOrderId.remove(indexx)
                        artworksToRemove.clear()
                        cycles += 1
                        i = 0
                    else :
                        i += 1
                    nRep -= 1
                
                orderId = printableArtworksOrderId[i]
                #print(f"orderId = {orderId}")
            
        if (returnOrder) :
            return (orderId, availableArtworks[orderId])
        else :
            return availableArtworks[orderId]
                
    def IterateOverArtworks(self, cardName, fromTemp = True) :
        inputString = (f"Continue ? (q : quit, *else* : yes -> Continue) => ")
        npt = ""
        i = 1
        while (npt != "q"):
            display.clear_output(wait=False)
            print(f"Iteration #{i} :")
            plt.imshow(mpimg.imread(self.GetArtworkPath(cardName, fromTemp)["path"]))
            plt.show()
            npt = input(inputString)
            i += 1
    
    
    def GetCardArtworkPathSelectionDisabled(self, name, artworksUrls) :
        filename = ""
        if (len(artworksUrls) == 1) :
            filename = name
        else : 
            filename = f"{name}-v1"
        filename += ".jpg"
        valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
        filename = ''.join(c for c in filename if c in valid_chars)
        filename = filename.replace(' ','_') # I don't like spaces in filenames.
        
        picturePath = self.artworksDirectoryPath + "\\" + filename
        
        if (not os.path.isfile(picturePath)) :
            print(f"Creating '{picturePath}'...")

            response = req.get(artworksUrls[0])
            file = open(picturePath, "wb")
            file.write(response.content)
            file.close()
        
        return picturePath
    
    def InitArtworkSelector(self, resetTempArtworkOrder = False) :
        self.currentCardArtwordOrder = dict()
        self.tempArtworkOrder = self.df[["artworksUrls"]]
        if ((not resetTempArtworkOrder) and ("artworks" in self.df.columns)) :
            self.tempArtworkOrder.loc[:, "order"] = self.df["artworks"]
        else :
            #self.df["artworks"] = np.nan
            self.tempArtworkOrder["order"] = np.nan
        #self.pltFig = plt.figure(figsize=(30,10))
        
    def SaveArtworkSelection(self) :
        self.df["artworks"] = self.tempArtworkOrder["order"]
            
    def DownloadArtwork(self, cardName, artworkUrl, onlyOneArtwork = True) :
        filename = ""
        if (onlyOneArtwork) : #len(artworksUrls) == 1) :
            filename = cardName

            picturePath = self.artworksDirectoryPath + "\\" + filename + ".jpg"

            if (not os.path.isfile(picturePath)) :
                print(f"Creating '{picturePath}'...")

                response = req.get(artworkUrl)
                file = open(picturePath, "wb")
                file.write(response.content)
                file.close()

            return picturePath
        else : 
            version = 0
            while True :
                version += 1
                filename = f"{cardName}-v{version}"
                filename += ".jpg"
                valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
                filename = ''.join(c for c in filename if c in valid_chars)
                filename = filename.replace(' ','_') # I don't like spaces in filenames.

                picturePath = self.artworksDirectoryPath + "\\" + filename

                if (not os.path.isfile(picturePath)) :
                    print(f"Creating '{picturePath}'...")

                    response = req.get(artworkUrl)
                    file = open(picturePath, "wb")
                    file.write(response.content)
                    file.close()

                    return picturePath
        
            
    def InitOrder(self, artworkUrls = []) : 
        order = dict()
        if (len(artworkUrls) == 0) :
            order[1] = {"path" : "", "url" : "", "printLimit" : 0}
        else :
            for i in range(len(artworkUrls)) :
                order[i+1] = {"path" : "", "url" : artworkUrls[i], "printLimit" : 0}
        return order
    
    def PrintOneArtwork(self, cardName, artworkPath, artworkUrl, ax, numberOfArtworks = 1, printArtwork = False) :
        if (artworkPath == "") :
            artworkPath = self.DownloadArtwork(cardName, artworkUrl, (numberOfArtworks == 1))
        img = mpimg.imread(artworkPath)
        ax.imshow(img)
        ax.axis("off")
        if (printArtwork) :
            plt.show()
            
        return artworkPath

            
    def PrintEveryArtworkForRow(self, row, artworksOrder = None, clearDisplay = False) :
        if (artworksOrder is None) :
                artworksOrder = self.GetArtworksOrder(row.name)
        
        if ((type(artworksOrder) != dict) and (np.isnan(artworksOrder))) :
            artworksOrder = self.InitOrder(row["artworksUrls"])
            
        artworksAmount = len(artworksOrder.keys())
        """for orderId in artworksOrder.keys() :
            self.PrintOneArtwork(artworksOrder[orderId])"""
        
        plt.figure(figsize=(30,30))
        nbOfArtwork = len(artworksOrder.keys())
        itemsPerLine = 4#int(np.sqrt(nbOfArtwork)) + 1
        if (itemsPerLine < nbOfArtwork) :
            itemsPerColon = nbOfArtwork // itemsPerLine + (1 if (nbOfArtwork % itemsPerLine) != 0 else 0)
        else :
            itemsPerLine = nbOfArtwork
            itemsPerColon = 1
        index = 0
        
        if (nbOfArtwork == 1) :
            artworksOrder[1]["path"] = self.DownloadArtwork(row.name,  artworksOrder[1]["url"], onlyOneArtwork = True)
            print(artworksOrder[1]["path"])
        else :
            for orderId in sorted(artworksOrder.keys()) :
                artwork = artworksOrder[orderId]
                ax = plt.subplot2grid((itemsPerLine, itemsPerLine), (index // itemsPerLine, index % itemsPerLine))
                printLimit = artwork["printLimit"]
                if (printLimit == 0) :
                    printLimit = "∞"
                elif (printLimit < 0) :
                    printLimit = 0
                ax.set_title(f"{orderId} (x {printLimit})", fontsize = 20)
                newArtworkPath = self.PrintOneArtwork(row.name, artwork["path"], artwork["url"], ax, artworksAmount)
                artworksOrder[orderId]["path"] = newArtworkPath
                index += 1

            if (clearDisplay) :
                display.clear_output(wait=True)
            plt.show()  
        
        return artworksOrder
    
    def AssignArtworksOrder(self, cardName, artworksOrder) :
        self.tempArtworkOrder.loc[cardName, "order"] = [artworksOrder]
    def GetArtworksOrder(self, cardName, fromTemp = True) :
        if (fromTemp) :
            df = self.tempArtworkOrder
        else :
            df = self.df
        if (cardName in df.index) :
            if (fromTemp) :
                artworksOrder = df.loc[cardName, "order"]
            else :
                artworksOrder = df.loc[cardName, "artworks"]
            if (type(artworksOrder) == list) :
                artworksOrder = artworksOrder[0]
            return artworksOrder
        else :
            #self.PrintError("GetArtworksOrder(...) -> No entry in self.tempArtworkOrder", f"(name = {cardName})")
            return None

    def SelectArtworkForRow(self, row, overwriteByDefault = False, unlimitedPrintsFirst = True) :        
        artworksOrder = self.GetArtworksOrder(row.name)
        artworkOrderAlreadyHasValue = not ((type(artworksOrder) != dict) and (np.isnan(artworksOrder)))
        if (artworkOrderAlreadyHasValue) :
            if (len(row["artworksUrls"]) == 1) :
                return
            if (not overwriteByDefault) :
                inputString = (f"There is already a value for {row.name}, skip it ? (n : no -> Overwrite, *else* : yes -> Skip) => ")
                npt = input(inputString)
                #display.clear_output(wait=True)
                if (npt != "n"):
                    return                                               
        
        artworksOrder = self.PrintEveryArtworkForRow(row, artworksOrder, clearDisplay = False)
        if (len(artworksOrder.keys()) == 1) :
            self.tempArtworkOrder.loc[row.name, "order"] = [artworksOrder]
            return
        
        noArtworkIsUnlimited = True
        for orderId in artworksOrder.keys() :
            printLimit = artworksOrder[orderId]["printLimit"]
            inputString = (f"Change print limit of artwork #{orderId} ? (current : {printLimit})\n" + 
                           f"(0 : unlimited, -1 : never, [1-...] : amount, *else* : leave as is) => ")
            npt = input(inputString)
            if (npt == "q"):
                self.AssignArtworksOrder(row.name, artworksOrder)
                return
            try :
                integer = int(npt)
                artworksOrder[orderId]["printLimit"] = integer
            except :
                print("")
            if (artworksOrder[orderId]["printLimit"] == 0) :
                noArtworkIsUnlimited = False
                
        while (noArtworkIsUnlimited) :
            inputString = ("One artwork must be unlimited, which one ? => ")
            npt = input(inputString)
            if (npt == "q"):
                self.AssignArtworksOrder(row.name, artworksOrder)
                return
        
            try :
                integer = int(npt)
                if (integer in artworksOrder.keys()) :
                    artworksOrder[integer]["printLimit"] = 0
                    noArtworkIsUnlimited = False
                else :
                    print(f"There is no Artwork #{integer}, only {list(artworksOrder.keys())}...")
            except :
                print("")
        
        newOrderPositive = 1
        newOrderNegative = -1
        newArtworksOrder = dict()
        unlimitedList = list()
        for orderId in artworksOrder.keys() :
            printLimit = artworksOrder[orderId]["printLimit"]
            if (printLimit > 0) :
                newArtworksOrder[newOrderPositive] = artworksOrder[orderId]
                newOrderPositive += 1
            elif (printLimit < 0) :
                newArtworksOrder[newOrderNegative] = artworksOrder[orderId]
                newOrderNegative -= 1   
            else : # == 0
                unlimitedList.append(artworksOrder[orderId])
                
                
        if (unlimitedPrintsFirst and newOrderPositive > 1) :
            for i in range(1, newOrderPositive) :
                reverseI = newOrderPositive - i
                newArtworksOrder[reverseI + len(unlimitedList)] = newArtworksOrder[reverseI]
            newOrderPositive = 1
            
        for ao in unlimitedList :
            newArtworksOrder[newOrderPositive] = ao
            newOrderPositive += 1
            
        artworksOrder = newArtworksOrder
        
        self.PrintEveryArtworkForRow(row, artworksOrder, clearDisplay = False)
        #print(artworksOrder)
        #print(self.tempArtworkOrder.at[row.name, "order"])
        self.AssignArtworksOrder(row.name, artworksOrder)
        
        inputString = (f"Ok ? (n : no -> Rechose, *else* : yes -> Continue) => ")
        npt = input(inputString)
        display.clear_output(wait=True)
        if (npt == "n"):
            self.SelectArtworkForRow(row, True)
            
    def SelectArtworkForAllDf(self, df = None, overwriteByDefault = False, unlimitedPrintsFirst = True) :
        if ((type(df) != pd.DataFrame) and (df is None)) :
            df = self.df
        df.apply(lambda x : ygoCardInfo.SelectArtworkForRow(x, overwriteByDefault, unlimitedPrintsFirst), axis = 1)
        self.df.loc[:, "artworks"] = self.tempArtworkOrder["order"]
        
        

In [163]:
class YGOProDeckAPICardInfos(YuGiOhCardInfos):
    
    def GetCardTypeAsEnum(self, frameType, race) :
        pendulumString = "pendulum"
        if (pendulumString in frameType) :
            frameType = frameType.replace(pendulumString, "")
            frameType = frameType.replace("__", "_")
            frameType = frameType.strip("_")
        if (frameType.lower() == "spell") :
            if (race =="Normal") :
                return CardType.SpellNormal
            elif (race == "Field") :
                return CardType.SpellField
            elif (race == "Ritual") :
                return CardType.SpellRitual
            elif (race == "Equip") :
                return CardType.SpellEquip
            elif (race == "QuickPlay") :
                return CardType.SpellQuickPlay
            elif (race == "Continuous") :
                return CardType.SpellContinuous
        elif (frameType.lower() == "trap") :
            if (race == "Normal") :
                return CardType.TrapNormal
            elif (race == "Counter") :
                return CardType.TrapCounter
            elif (race == "Continuous") :
                return CardType.TrapContinuous
        else :#if (frameType in ["normal", "effect", "ritual", "fusion", "synchro", "xyz", "link"]) :
            if (frameType == "normal") :
                return CardType.MonsterNormal
            elif (frameType == "effect") :
                return CardType.MonsterEffect
            elif (frameType == "ritual") :
                return CardType.MonsterRitual
            elif (frameType == "fusion") :
                return CardType.MonsterFusion
            elif (frameType == "synchro") :
                return CardType.MonsterSynchro
            elif (frameType == "xyz") :
                return CardType.MonsterXyz
            elif (frameType == "link") :
                return CardType.MonsterLink
            
        self.PrintError(f"GetCardTypeAsEnum(frameType = {frameType}, race = {race}) = Cannot handle this combination.")
        return None
            
    def GetMonsterRaceAndArchetypeList(self, monsterRace, monsterType, cardText, isPendulum = False) :
        result = [monsterRace]
        if (monsterType == "Fusion Monster") :
            result.append("Fusion")
        elif (monsterType == "Synchro Monster") :
            result.append("Synchro")
        elif (monsterType == "Synchro Tuner Monster") :
            result.append("Synchro")
            result.append("Tuner")
        elif (monsterType == "XYZ Monster") :
            result.append("XYZ")
        elif (monsterType == "Link Monster") :
            result.append("Link")
        
        #print(len(result))
        if (len(result) > 1) :
            if (isPendulum) :
                result.append("Pendulum")
            if ("\n" in cardText) :
                result.append("Effect")
            else :
                result.append("Normal")
            return result
            
        if (monsterType == "Ritual Effect Monster") :
            result.append("Ritual")
            result.append("Effect")
            return result
        elif (monsterType == "Ritual Monster") :
            result.append("Ritual")
            result.append("Normal")
            return result
        
        for archetype in ["Gemini", "Spirit", "Toon", "Union"] :
            if (archetype in monsterType) :
                result.append("Toon")
                result.append("Effect")
                return result
            #else : 
            #    print(f"Not a {archetype}...")
            
        for archetype in ["Flip", "Tuner"] :
            if (archetype in monsterType) :
                result.append(archetype)
        
        if (isPendulum) :
            result.append("Pendulum")
            
        if ("Effect" in monsterType) :
            result.append("Effect")
        elif ("Normal" in monsterType) :
            result.append("Normal")
        
        return result
        
        
    def PrintError (self, errorText, inputText = None) :
        return super().PrintError(errorText, inputText, "YGOProDeckAPICardInfos")
    
    fieldCorrespondance = {
        "id"                   : "id",
        "name"                 : "name",
        #"type"                 : "type",
        #"frameType"            : "frame_type",
        "desc"                 : "card_text",
        "race"                 : "race",
        "name_en"              : "name_en",
        "atk"                  : "atk",
        "def"                  : "def",
        "level"                : "level",
        "attribute"            : "attribute",
        "linkval"              : "linkval",
        "pendulum_effect"      : "pendulum_effect",
        "pendulum_scale"       : "pendulum_scale"
    }
        
    def GetCardInfoByName(self, name) :
        name = name.strip()
        if (name in self.aliases.keys()) :
            #print(f"GetCardInfoByName(name = {name}) -> Changed name to '{self.aliases[name]}'")       
            if (not self.df.empty and self.aliases[name] in self.df.index) :
                print(f"GetCardInfoByName(name = {name}) -> Already in Database (under the name '{self.aliases[name]}')")
                return self.df.loc[self.aliases[name]]
            name = self.aliases[name]  
        else :            
            if (not self.df.empty and name in self.df.index) :
                print(f"GetCardInfoByName(name = {name}) -> Already in Database")
                return self.df.loc[name]
            
        try :
            url = f"https://db.ygoprodeck.com/api/v7/cardinfo.php?name={name}"
            if (self.language != "en") :
                url += f"&language={self.language}"
            r = req.get(url)
        except :
            self.PrintError(f"GetCardInfoByName(name = {name}) -> API error : {sys.exc_info()[0]}", name)
            return None
        if ("error" in r.json().keys()) :
            self.PrintError(f"GetCardInfoByName(name = {name}) -> API error : '{r.json()['error']}'", name)
            return None 
        infoJson = r.json()["data"][0]
        cardInfo = dict()
        infoToCollect = set(("id", "name", "type", "frameType", "desc", "race", 
                             "name_en", "atk", "def", "level", "attribute", "linkval" ,
                            "pendulum_effect", "pendulum_scale"))
        for fieldName in infoToCollect :
            if (fieldName in infoJson.keys()) : 
                cardInfo[fieldName] = infoJson[fieldName]
            else :
                cardInfo[fieldName] = None
                
        # Attribute
        if (cardInfo["attribute"] == None) :
            cardInfo["attribute"] = cardInfo["frameType"]
        
        # Artworks
        artworksUrls = []
        for alternateArt in infoJson["card_images"] :
            artworksUrls.append(alternateArt["image_url_cropped"])
        
        # Link
        linkmarkers = None
        if "linkmarkers" in infoJson :
            linkmarkers = self.GetLinkArrowsAsAListOfEnum(infoJson["linkmarkers"])
                
        # Pendulum
        cardType = cardInfo["type"]
        pendulumStringLowercase = "pendulum".lower()
        isPendulum = pendulumStringLowercase in cardType.lower()
        if (isPendulum) :
            startIndex = cardType.lower().index(pendulumStringLowercase)
            newCardType = ""
            if (startIndex > 0) :
                newCardType = cardType[:startIndex].strip()
            if ((startIndex + len(pendulumStringLowercase)) < (len(cardType) - 1)) :
                lastPart = cardType[startIndex + len(pendulumStringLowercase):].strip()
                newCardType = f"{newCardType.strip()} {lastPart.strip()}".strip()
            #cardInfo["type"] = newCardType
            if "pend_desc" in infoJson :
                cardInfo["pendulum_effect"] = infoJson["pend_desc"]
            if "monster_desc" in infoJson :
                cardInfo["desc"] = infoJson["monster_desc"]
            if "scale" in infoJson :
                cardInfo["pendulum_scale"] = infoJson["scale"]
        else :
            cardInfo["pendulum_effect"] = None
            cardInfo["pendulum_scale"] = None
            
        #cardInfo["is_pendulum"] = isPendulum
        
        # Types rework
        if ("monster" in cardInfo["type"].lower()) :
            cardTypes = self.GetMonsterRaceAndArchetypeList(cardInfo["race"], cardInfo["type"], cardInfo["desc"],
                                                            "pendulum" in cardInfo["frameType"].lower())
            cardInfo["test1"] = cardInfo["frameType"]
            cardInfo["test2"] = cardInfo["race"]
            cardInfo["test3"] = cardInfo["type"]
            cardInfo["type"] = "Monster"
        else :
            cardTypes = [cardInfo["race"]]
            cardInfo["type"] = cardInfo["type"].replace("Card", "").strip()
        
        cardInfo["card_type_enum"] = self.GetCardTypeAsEnum(cardInfo["frameType"], cardInfo["race"])
            
        cardInfo.pop("frameType")
        cardInfo.pop("race")
            
        # Aliases
        if (name != cardInfo["name"]) :
            self.aliases[name] = cardInfo["name"]
            if (not self.df.empty and cardInfo["name"] in self.df.index) :
                print(f"GetCardInfoByName(name = {name}) -> Already in Database (under the name '{cardInfo['name']}')")
                return self.df.loc[cardInfo["name"]]
            name = cardInfo["name"]
        
        
        #print(cardInfo)
        cardInfo = self.ConvertAllFields(cardInfo) #use self.fieldCorrespondance
        print(cardInfo)
        #cardInfo["artworksUrls"] = np.array(artworksUrls)
        #print(cardInfo)
        #cardInfoAsDf = pd.DataFrame.from_dict(cardInfo)
        cardInfoAsDf = pd.Series(cardInfo, name=name).to_frame().T
        #cardInfoAsDf.at[0, "artworksUrls"] = np.array(artworksUrls)
        #print(cardInfoAsDf)
        self.df = pd.concat([self.df, cardInfoAsDf])#, ignore_index=True)
        self.df.at[name, "artworksUrls"] = artworksUrls
        self.df.at[name, "linkmarkers"] = linkmarkers
        self.df.at[name, "race"] = cardTypes
        #self.df.add(cardInfo)
        print(f"GetCardInfoByName(name = {name}) -> Added to Database")
        return self.df.loc[name]

In [190]:
class CardInfoToPSD :
    def __init__(self,
                 language = "fr",
                 ygoPsd = None,
                 ygoCardInfo = None,
                 translationManager = None,
                 keepThePsds = False) :
        if (ygoPsd is None) :
            self.ygoPsd = YuGiOhCardPsd()
        else :
            self.ygoPsd = ygoPsd
        if (ygoCardInfo is None) :
            self.ygoCardInfo = YGOProDeckAPICardInfos(language=language)
        else :
            self.ygoCardInfo = ygoCardInfo
        if (translationManager is None) :
            self.translationManager = YugiohTranslation(language)
        else :
            self.translationManager = translationManager
        self.keepThePsds = keepThePsds
        #self.InitMonsterRaceTranslations()
        self.printedCards = pd.DataFrame(columns = ["nameAndArtwork", 
                                                    "cardName", 
                                                    "collectorNumber", 
                                                    "cardPath", 
                                                    "comments"])
        self.printedCards.set_index("nameAndArtwork", inplace = True)
        self.language = language
            
    def GetMonsterTypeString(self, cardRace): #cardText, monsterType = None, isPendulum = False) :
        """if (monsterType is None) :
            if (self.ygoPsd.cardType is None) :
                print(f"ERROR : CardInfoToPSD.GetMonsterType(...) : monsterType and self.ygoPsd.cardType are both None.")
                return None
            monsterType = self.ygoPsd.cardType
        archetypes = self.GetArchetypeList(monsterType, cardText, isPendulum)"""
        archetypes = cardRace
        translatedArchetypes = ""
        for component in archetypes :
            if (translatedArchetypes == "") :
                translatedArchetypes += f"{self.translationManager.TranslateMonsterRace(component)}"
            else :
                translatedArchetypes += f"/{self.translationManager.TranslateMonsterArchetype(component)}"
        return f"[{translatedArchetypes}]"
            
        
    """def GetArchetypeList(self, monsterType, cardText, isPendulum = False) :
        result = []
        if (monsterType == "Fusion Monster") :
            result.append("Fusion")
        elif (monsterType == "Synchro Monster") :
            result.append("Synchro")
        elif (monsterType == "Synchro Tuner Monster") :
            result.append("Synchro")
            result.append("Tuner")
        elif (monsterType == "XYZ Monster") :
            result.append("XYZ")
        elif (monsterType == "Link Monster") :
            result.append("Link")
        
        #print(len(result))
        if (len(result) != 0) :
            if (isPendulum) :
                result.append("Pendulum")
            if ("\n" in cardText) :
                result.append("Effect")
            else :
                result.append("Normal")
            return result
            
        if (monsterType == "Ritual Effect Monster") :
            return ["Ritual", "Effect"]
        elif (monsterType == "Ritual Monster") :
            return ["Ritual", "Normal"]
        
        for archetype in ["Gemini", "Spirit", "Toon", "Union"] :
            if (archetype in monsterType) :
                return [archetype, "Effect"]
            #else : 
            #    print(f"Not a {archetype}...")
            
        for archetype in ["Flip", "Tuner"] :
            if (archetype in monsterType) :
                result.append(archetype)
        
        if (isPendulum) :
            result.append("Pendulum")
            
        if ("Effect" in monsterType) :
            result.append("Effect")
        elif ("Normal" in monsterType) :
            result.append("Normal")
        
        return result"""
    
    def StorePrint(self, cardName, artworkOrder, collectorNumber, cardPath, comments = None) :
        print("### CardInfoToPSD.StorePrint() : NOT SUPPORTED ###")
        return False
    ### NOT REACHABLE ###
        index = f"{cardName}-V{order}"
        if (index in self.printedCards.keys()) :
            return
        cardInfo = {
            #"nameAndArtwork" = index,
            "cardName" : cardName,
            "collectorNumber" : collectorNumber,
            "cardPath" : cardPath,
            "comments" : comments
        }
        cardInfoAsDf = pd.Series(cardInfo, name=index).to_frame().T
        #cardInfoAsDf.at[0, "artworksUrls"] = np.array(artworksUrls)
        #print(cardInfoAsDf)
        self.printedCards = pd.concat([self.printedCards, cardInfoAsDf])
     
    def MakeCardFromName(self, name, keepThePsd = False) :
        cardInfo = self.ygoCardInfo.GetCardInfoByName(name)
        if (cardInfo is None) :
            return False
        
        cardType = cardInfo["card_type_enum"]#self.GetCardType(cardInfo["frameType"], cardInfo["race"])
        if (cardType is None) :
            return False
            
        cardArchetype = cardType // 10

        self.ygoPsd.MakeCardType(cardType)

        self.ygoPsd.SetCardName(cardInfo["name"])
        attribute = cardInfo["attribute"]

        #if (attribute is None) : # Spell/Traps
        #    attribute = cardInfo[""]
        self.ygoPsd.SetAttribute(attribute, self.language)

        self.ygoPsd.SetSerialNumber(cardInfo["id"])
        
        isPendulum = (cardInfo["pendulum_effect"] is not None) and (cardInfo["pendulum_scale"] is not None)
          
        if (isPendulum) :
            collectorNumber = self.ygoPsd.SetCollectorNumber(layerName = LayerNames.SetNumberPendulum) 
        else :
            self.ygoPsd.SetCardText(cardInfo["card_text"])
            collectorNumber = self.ygoPsd.SetCollectorNumber() 

        #artworkPath = self.ygoCardInfo.GetCardArtworkPathSelectionDisabled(cardInfo["name"], cardInfo["artworksUrls"])
        artworkPath = self.ygoCardInfo.GetArtworkPath(cardInfo["name"])

        if (cardArchetype == CardType.Monster) :
            if (cardType == CardType.MonsterLink) :
                self.ygoPsd.SetLinkArrowsVisibility(cardInfo["linkmarkers"])
                self.ygoPsd.SetAtkAndLinkValue(f"{cardInfo['atk']}", f"{cardInfo['linkval']}")
            else :
                self.ygoPsd.SetAtkAndDef(f"{cardInfo['atk']}", f"{cardInfo['def']}")
                
            #raceAsString = self.translationManager.TranslateMonsterRace(cardInfo["race"])
            #print(f"{cardInfo['desc'] = }|{cardInfo['type'] = }")
            #typesAsString = self.GetMonsterType(cardInfo["card_text"], cardInfo["type"], isPendulum)
            monsterType = self.GetMonsterTypeString(cardInfo["race"])#f"[{raceAsString}{typesAsString}]"
                 
            pendulumSize = None
            if (isPendulum) :
                pendulumSize = self.ygoPsd.GetPendulumSize()
                self.ygoPsd.ChangePendulumVisibility(True)
                self.ygoPsd.SetPendulumCardText(cardInfo["card_text"], cardInfo["pendulum_effect"], pendulumSize, cardType)
                self.ygoPsd.SetPendulumScales(cardInfo["pendulum_scale"], pendulumSize)
                self.ygoPsd.SetPendulumMonsterRace(monsterType, pendulumSize)
            elif ((cardInfo["pendulum_effect"] is not None) or (cardInfo["pendulum_scale"] is not None)) :
                pendulumEffect = cardInfo["pendulum_effect"]
                pendulumScale = cardInfo["pendulum_scale"]
                print(f"ERROR : CardInfoToPSD.MakeCardFromName({name = }) -> {pendulumEffect = } & {pendulumScale = } (One of them is None).")
                return False
            else :
                self.ygoPsd.ChangePendulumVisibility(False)
                self.ygoPsd.SetMonsterType(monsterType)
                
            isXYZ = "XYZ" in cardInfo["race"]
            self.ygoPsd.ChangeTextsAndPictureAndLevel(artworkPath, cardInfo["level"], isXYZ, pendulumSize)
            #Not Supported nor Useful with current PSD
            #self.ygoPsd.SetMonsterTypeRightBracket()
        else :
            self.ygoPsd.ChangePendulumVisibility(False)
            self.ygoPsd.ChangeTextsAndPicture(artworkPath)

        if (keepThePsd or self.keepThePsds) :

            self.ygoPsd.SavePsd()

        cardPath = self.ygoPsd.RenderPng()
        
        self.StorePrint(cardInfo["name"],
                        None, #artworkOrder  
                        collectorNumber, 
                        cardPath)
        
        return True
    
    def GetEveryCardForYgoOmegaDeckList(self, path = r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\source.txt") :
        decomposer = re.compile(r'^(?P<qty>\d+)\s(?P<name>.+)$')
        cardsDone = set()
        with open(path, encoding='utf-8') as file:
            line = file.readline().strip("\n") 
            while (line != "") :
                searchResult = decomposer.search(line)
                if (searchResult is not None) :
                    qty = searchResult.group("qty")    
                    name = searchResult.group("name") 
                    if (name not in cardsDone) :
                        cardsDone.add(name)
                        self.MakeCardFromName(name)
                line = file.readline().strip("\n") 

In [191]:
ygopsd = YuGiOhCardPsd(printDebug=False)
ygoCardInfo = YGOProDeckAPICardInfos(language = "fr")
cardInfoToPsd = CardInfoToPSD("fr", ygopsd, ygoCardInfo, keepThePsds=True)

In [193]:
cardInfoToPsd.GetEveryCardForYgoOmegaDeckList()

GetCardInfoByName(name = Dragon Toon aux Yeux Bleus) -> Already in Database
### CardInfoToPSD.StorePrint() : NOT SUPPORTED ###
{'linkval': None, 'atk': 2000, 'name': 'Magicienne des Ténèbres Toon', 'attribute': 'DARK', 'name_en': 'Toon Dark Magician Girl', 'pendulum_scale': None, 'type': 'Monster', 'id': 90960358, 'level': 6, 'pendulum_effect': None, 'def': 1700, 'test1': 'effect', 'test2': 'Spellcaster', 'test3': 'Toon Monster', 'card_type_enum': <CardType.MonsterEffect: 35>, 'card_text': 'Ni Invocable Normalement ni Posable Normalement. Doit d\'abord être Invoquée Spécialement (depuis votre main) en Sacrifiant 1 monstre, tant que vous contrôlez "Monde des Toons". Si "Monde des Toons" sur le Terrain est détruit, détruisez cette carte. Peut attaquer directement votre adversaire, sauf s\'il contrôle un monstre Toon, auquel cas cette carte doit cibler un monstre Toon avec ses attaques. Gagne 300 ATK pour chaque "Magicien Sombre" et "Magicien du Chaos Sombre" dans les Cimetières.'}
GetCar

### CardInfoToPSD.StorePrint() : NOT SUPPORTED ###
{'linkval': None, 'atk': 900, 'name': 'Sorcier Masqué Toon', 'attribute': 'DARK', 'name_en': 'Toon Masked Sorcerer', 'pendulum_scale': None, 'type': 'Monster', 'id': 16392422, 'level': 4, 'pendulum_effect': None, 'def': 1400, 'test1': 'effect', 'test2': 'Spellcaster', 'test3': 'Toon Monster', 'card_type_enum': <CardType.MonsterEffect: 35>, 'card_text': 'Ne peut pas attaquer le tour où elle est Invoquée. Si "Monde des Toons" sur le Terrain est détruit, détruisez cette carte. Tant que vous contrôlez "Monde des Toons" et que votre adversaire ne contrôle aucun monstre Toon, cette carte peut attaquer directement votre adversaire. Si cette carte inflige des dommages de combat à votre adversaire : piochez 1 carte.'}
GetCardInfoByName(name = Sorcier Masqué Toon) -> Added to Database
Creating 'C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\Artworks\Sorcier_Masqu_Toon.jpg'...
### CardInfoToPSD.StorePrint() : NOT SUPPORTED ###
{'linkval': None, 'at

### CardInfoToPSD.StorePrint() : NOT SUPPORTED ###
{'linkval': None, 'atk': None, 'name': 'Main Comique', 'attribute': 'spell', 'name_en': 'Comic Hand', 'pendulum_scale': None, 'type': 'Spell', 'id': 33453260, 'level': None, 'pendulum_effect': None, 'def': None, 'card_type_enum': <CardType.SpellEquip: 14>, 'card_text': 'Si vous contrôlez "Monde des Toons", équipez cette carte à un monstre de l\'adversaire. Prenez son contrôle. Il est traité comme un monstre Toon. Si votre adversaire ne contrôle aucun monstre Toon, il peut attaquer directement votre adversaire. Si "Monde des Toons" n\'est pas sur le Terrain, détruisez cette carte.'}
GetCardInfoByName(name = Main Comique) -> Added to Database
Creating 'C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\Artworks\Main_Comique.jpg'...
### CardInfoToPSD.StorePrint() : NOT SUPPORTED ###
ERROR : YGOProDeckAPICardInfos.GetCardInfoByName(name = Tables des Matières Toon) -> API error : 'No card matching your query was found in the database. Please see 

### CardInfoToPSD.StorePrint() : NOT SUPPORTED ###
{'linkval': None, 'atk': None, 'name': 'Métavers', 'attribute': 'trap', 'name_en': 'Metaverse', 'pendulum_scale': None, 'type': 'Trap', 'id': 89208725, 'level': None, 'pendulum_effect': None, 'def': None, 'card_type_enum': <CardType.TrapNormal: 21>, 'card_text': 'Prenez 1 Magie de Terrain depuis votre Deck, et soit activez-la soit ajoutez-la à votre main.'}
GetCardInfoByName(name = Métavers) -> Added to Database
Creating 'C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\Artworks\Mtavers.jpg'...
### CardInfoToPSD.StorePrint() : NOT SUPPORTED ###
{'linkval': None, 'atk': 0, 'name': 'Le Renoncé aux Milles Yeux', 'attribute': 'DARK', 'name_en': 'Thousand-Eyes Restrict', 'pendulum_scale': None, 'type': 'Monster', 'id': 63519819, 'level': 1, 'pendulum_effect': None, 'def': 0, 'test1': 'fusion', 'test2': 'Spellcaster', 'test3': 'Fusion Monster', 'card_type_enum': <CardType.MonsterFusion: 33>, 'card_text': '"Le Renoncé" + "Idole aux Milles Yeux"\n

In [168]:
#cardInfoToPsd.MakeCardFromName("Obelisk, le tourmenteur")
#cardInfoToPsd.MakeCardFromName("Arsenal Divin AA-ZEUS - Tonnerre du Ciel")
#cardInfoToPsd.MakeCardFromName("Magicien Sombre")
#cardInfoToPsd.MakeCardFromName("Numéro 39 : Utopie")
#cardInfoToPsd.MakeCardFromName("Crâne Invoqué Toon")
#cardInfoToPsd.MakeCardFromName("Dragon Cyber Ultime")
#cardInfoToPsd.MakeCardFromName("Cylindre Magique")
#cardInfoToPsd.MakeCardFromName("Charité Gracieuse")
#cardInfoToPsd.MakeCardFromName("Mausolée de l'empereur")
#cardInfoToPsd.MakeCardFromName("Dragon Pare-Feu")
#cardInfoToPsd.MakeCardFromName("Le Renoncé")
#cardInfoToPsd.MakeCardFromName("Empereur du Chaos, le Dragon de l'Armageddon")
cardInfoToPsd.MakeCardFromName("Bickuribox")

{'linkval': None, 'atk': 2300, 'name': 'Bickuribox', 'attribute': 'DARK', 'name_en': 'Bickuribox', 'pendulum_scale': None, 'type': 'Monster', 'id': 25655502, 'level': 7, 'pendulum_effect': None, 'def': 2000, 'test1': 'fusion', 'test2': 'Fiend', 'test3': 'Fusion Monster', 'card_type_enum': <CardType.MonsterFusion: 33>, 'card_text': '"Clown Grossier" + "Clown de Rêve"'}
GetCardInfoByName(name = Bickuribox) -> Added to Database
### CardInfoToPSD.StorePrint() : NOT SUPPORTED ###


True

In [51]:
ygopsd.GetValueAsFormattedString(2500, 4)

'2500'

In [52]:
ygopsd.SetCardName("Le Renoncé aux Milles Yeux")
ygopsd.ChangeAllTexts()

DEBUG : YuGiOhCardPsd.ChangeAllTextColors() -> Setting color 'Black' to layer 'Set Number (General)'...
DEBUG : YuGiOhCardPsd.ChangeAllTextColors() -> Setting color 'Black' to layer 'Edition'...
DEBUG : YuGiOhCardPsd.ChangeAllTextColors() -> Setting color 'Black' to layer 'Code'...
DEBUG : YuGiOhCardPsd.ChangeAllTextColors() -> Setting color 'Black' to layer 'Copyright'...
DEBUG : YuGiOhCardPsd.ChangeAllTextValues() -> Setting 'ÉDITION LIMITÉE' to layer 'Edition'...
DEBUG : YuGiOhCardPsd.ChangeAllTextValues() -> Setting 'Le Renoncé aux Milles Yeux' to layer 'Card Name (Black)'...
DEBUG : YuGiOhCardPsd.ChangeAllTextValues() -> Setting '91842653' to layer 'Code'...
Ni Invocable Normalement ni Posable Normalement. Doit d'abord être Invoquée Spécialement (depuis votre main) en Sacrifiant 1 monstre, tant que vous contrôlez "Monde des Toons". Ne peut pas attaquer le tour où elle est Invoquée Spécialement. Vous devez payer 500 LP pour déclarer une attaque avec ce monstre. Si "Monde des Toons"

In [53]:
type(cardInfoToPsd.ygoCardInfo)

__main__.YGOProDeckAPICardInfos

In [54]:
for cardName in ygoCardInfo.df[(ygoCardInfo.df["frameType"] == "spell") | (ygoCardInfo.df["frameType"] == "trap")].index :
    cardInfoToPsd.MakeCardFromName(cardName)

KeyError: 'frameType'

In [155]:
ygoCardInfo.GetCardInfoByName("Magicien Sombre")
ygoCardInfo.GetCardInfoByName("Magicien du Temps")
ygoCardInfo.GetCardInfoByName("Bickuribox")
ygoCardInfo.GetCardInfoByName("Guerrier de la Route")
ygoCardInfo.GetCardInfoByName("Numéro 39 : Utopie")
ygoCardInfo.GetCardInfoByName("Soldat du Lustre Noir")
ygoCardInfo.GetCardInfoByName("Crâne Invoqué Toon")
ygoCardInfo.GetCardInfoByName("Hane-Hane")
ygoCardInfo.GetCardInfoByName("Dragon Cyber Ultime")

ygoCardInfo.GetCardInfoByName("Salamandra")
ygoCardInfo.GetCardInfoByName("Riryoku")
ygoCardInfo.GetCardInfoByName("Typhon d'Espace Mystique")
ygoCardInfo.GetCardInfoByName("Terre Embrasée")
ygoCardInfo.GetCardInfoByName("Rituel de la Noire Illusion")
ygoCardInfo.GetCardInfoByName("Mausolée de l'empereur")

ygoCardInfo.GetCardInfoByName("Chausse-Trape")
ygoCardInfo.GetCardInfoByName("Jugement Solennel")
ygoCardInfo.GetCardInfoByName("L'Incarnation d'Apophis")

ygoCardInfo.GetCardInfoByName("Terre Embrasee")

ygoCardInfo.df

GetCardInfoByName(name = Magicien Sombre) -> Already in Database
GetCardInfoByName(name = Magicien du Temps) -> Already in Database
GetCardInfoByName(name = Bickuribox) -> Already in Database
GetCardInfoByName(name = Guerrier de la Route) -> Already in Database
GetCardInfoByName(name = Numéro 39 : Utopie) -> Already in Database
GetCardInfoByName(name = Soldat du Lustre Noir) -> Already in Database
GetCardInfoByName(name = Crâne Invoqué Toon) -> Already in Database
GetCardInfoByName(name = Hane-Hane) -> Already in Database
GetCardInfoByName(name = Dragon Cyber Ultime) -> Added to Database
GetCardInfoByName(name = Salamandra) -> Already in Database
GetCardInfoByName(name = Riryoku) -> Already in Database
GetCardInfoByName(name = Typhon d'Espace Mystique) -> Already in Database
GetCardInfoByName(name = Terre Embrasée) -> Already in Database
GetCardInfoByName(name = Rituel de la Noire Illusion) -> Already in Database
GetCardInfoByName(name = Mausolée de l'empereur) -> Already in Database (

,id,desc,race,frameType,type,name_en,atk,def,level,attribute,artworksUrls,name
Magicien Sombre,46986414,Mage suprême en termes d'attaque et de défense.,Spellcaster,normal,Normal Monster,Dark Magician,2500,2100,7,DARK,[https://images.ygoprodeck.com/images/cards_cr...,Magicien Sombre
Magicien du Temps,71625222,Une fois par tour : vous pouvez jouer à pile o...,Spellcaster,effect,Effect Monster,Time Wizard,500,400,2,LIGHT,[https://images.ygoprodeck.com/images/cards_cr...,Magicien du Temps
Bickuribox,25655502,"""Clown Grossier"" + ""Clown de Rêve""",Fiend,fusion,Fusion Monster,Bickuribox,2300,2000,7,DARK,[https://images.ygoprodeck.com/images/cards_cr...,Bickuribox
Guerrier de la Route,2322421,"""Route Synchronique"" + 2 monstres non-Syntonis...",Warrior,synchro,Synchro Monster,Road Warrior,3000,1500,8,LIGHT,[https://images.ygoprodeck.com/images/cards_cr...,Guerrier de la Route
Numéro 39 : Utopie,84013237,2 monstres de Niveau 4\nLorsqu'un monstre décl...,Warrior,xyz,XYZ Monster,Number 39: Utopia,2500,2000,4,LIGHT,[https://images.ygoprodeck.com/images/cards_cr...,Numéro 39 : Utopie
Salamandra,32268901,Équipable uniquement à un monstre FEU. Il gagn...,Equip,spell,Spell Card,Salamandra,None,None,None,None,[https://images.ygoprodeck.com/images/cards_cr...,Salamandra
Riryoku,34016756,Ciblez 2 monstres face recto sur le Terrain ; ...,Normal,spell,Spell Card,Riryoku,None,None,None,None,[https://images.ygoprodeck.com/images/cards_cr...,Riryoku
Typhon d'Espace Mystique,5318639,Ciblez 1 Magie/Piège sur le Terrain ; détruise...,Quick-Play,spell,Spell Card,Mystical Space Typhoon,None,None,None,None,[https://images.ygoprodeck.com/images/cards_cr...,Typhon d'Espace Mystique
Terre Embrasée,24294108,Lorsque cette carte est activée : s'il y a des...,Continuous,spell,Spell Card,Burning Land,None,None,None,None,[https://images.ygoprodeck.com/images/cards_cr...,Terre Embrasée
Chausse-Trape,64697231,Activable uniquement lorsque votre adversaire ...,Normal,trap,Trap Card,Trap Dustshoot,None,None,None,None,[https://images.ygoprodeck.com/images/cards_cr...,Chausse-Trape


In [371]:
ygoCardInfo.SavePickle()

In [200]:
ygopsd.MakeCardType(CardType.MonsterNormal)
ygopsd.SetCardName("Magicien Sombre")
ygopsd.SetMonsterType("Magicien / Normal")
ygopsd.SetAtkAndDef("2500", "2000")
ygopsd.SetCardText("Mage suprême en terme d'attaque et de défense.")
ygopsd.SetCollectorNumber()
ygopsd.ChangeAllTexts()
ygopsd.SetMonsterTypeRightBracket()
ygopsd.psd.composite(force=True).save(r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\Kek\Test DM.png")

DEBUG : ChangeGroupOrLayerVisibility(Monster) -> 'Monster' is already visible.
DEBUG : ChangeGroupOrLayerVisibility(Spell/Trap) -> 'Spell/Trap' is already hidden.
DEBUG : ChangeGroupOrLayerVisibility(Token Text) -> 'Token Text' is already hidden.
DEBUG : ChangeGroupOrLayerVisibility(Card Text) -> 'Card Text' is already visible.
DEBUG : ChangeAllTexts() -> Setting 'Magicien Sombre' to layer 'Card Name V1'...
DEBUG : ChangeAllTexts() -> Setting 'Magicien / Normal' to layer 'Type/Subtype'...
DEBUG : ChangeAllTexts() -> Setting '2500' to layer 'ATK'...
DEBUG : ChangeAllTexts() -> Setting '2000' to layer 'DEF'...
DEBUG : ChangeAllTexts() -> Setting 'Mage suprême en terme d'attaque et de défense.' to layer 'Effect Card Lore'...
DEBUG : ChangeAllTexts() -> Setting 'CSTM-FR001' to layer 'Set ID'...


In [348]:
for cardName in ["Soldat du Lustre Noir", "Crâne Invoqué Toon", "Hane-Hane", 
                 "Dragon Cyber Ultime", "Typhon d'Espace Mystique", 
                 "Rituel de la Noire Illusion", "Mausolée de l'empereur"] :#cardInfoToPsd.ygoCardInfo.df.index :
    cardInfoToPsd.MakeCardFromName(cardName)
cardInfoToPsd.ygoCardInfo.SavePickle()

GetCardInfoByName(name = Soldat du Lustre Noir) -> Added to Database
WAAA
DEBUG : ChangeGroupOrLayerVisibility(Token Text) -> 'Token Text' is already hidden.
DEBUG : ChangeGroupOrLayerVisibility(Card Text) -> 'Card Text' is already visible.
Creating 'C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\Artworks\Soldat_du_Lustre_Noir.jpg'...
DEBUG : ChangeAllTexts() -> Setting 'Soldat du Lustre Noir' to layer 'Card Name V1'...
DEBUG : ChangeAllTexts() -> Setting 'Vous pouvez Invoquer Rituellement cette carte avec "Rituel du Lustre Noir".' to layer 'Effect Card Lore'...
DEBUG : ChangeAllTexts() -> Setting 'CSTM-FR012' to layer 'Set ID'...
DEBUG : ChangeAllTexts() -> Setting 'Warrior' to layer 'Type/Subtype'...
DEBUG : ChangeAllTexts() -> Setting '3000' to layer 'ATK'...
DEBUG : ChangeAllTexts() -> Setting '2500' to layer 'DEF'...
GetCardInfoByName(name = Crâne Invoqué Toon) -> Added to Database
DEBUG : ChangeGroupOrLayerVisibility(Monster) -> 'Monster' is already visible.
DEBUG : ChangeGroupOrLa

In [176]:
ygopsd.SetCardName("Le Renoncé aux Milles Yeux")
ygopsd.ChangeAllTexts()

DEBUG : ChangeAllTexts() -> Setting 'Le Renoncé aux Milles Yeux' to layer 'Card Name V1'...
DEBUG : ResizeAllTexts() -> Resizing text for layer 'Card Name V1'...
Bounds Layer Found => 'Card Name - Bounds'


In [714]:
limit = 312
#ygopsd = YuGiOhCardPsd()
layerName = "Effect Card Lore"
psd_api = ygopsd.PhotoshopApp.Open(ygopsd.path)

hierarchy = ygopsd.GetGroupOrLayerPathFromRootByName(layerName)
layer = psd_api
hierarchy.pop()
while (len(hierarchy) > 0) :
    childName = hierarchy.pop()
    #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
    layer = layer.Layers[childName]
boundsLayer = layer.parent.Layers["Effect Card Lore - Bounds"]
"""layer.TextItem.HorizontalScale = 100
while ((layer.bounds[2] - layer.bounds[0]) > limit) :
    layer.TextItem.HorizontalScale -= 10
while ((layer.bounds[2] - layer.bounds[0]) < limit) :
    layer.TextItem.HorizontalScale += 1"""
    

'layer.TextItem.HorizontalScale = 100\nwhile ((layer.bounds[2] - layer.bounds[0]) > limit) :\n    layer.TextItem.HorizontalScale -= 10\nwhile ((layer.bounds[2] - layer.bounds[0]) < limit) :\n    layer.TextItem.HorizontalScale += 1'

In [164]:
textLayer = layer
boundaries = boundsLayer.bounds 
limit = [2]

textLayer.TextItem.HorizontalScale = 100
textLayer.TextItem.VerticalScale = 100
#iteration = 0
while (not ygopsd.IsLayerInBounds(textLayer, boundaries, limit)) :
    textLayer.TextItem.HorizontalScale -= 10
    #iteration += 1
    """print(f"### Iteration n°{iteration}")
    print(f"Size = {layer.TextItem.Size}")
    print(f"HorizontalScale = {layer.TextItem.HorizontalScale}")
    print(f"VerticalScale = {layer.TextItem.VerticalScale}")"""
while (ygopsd.IsLayerInBounds(textLayer, boundaries, limit)) :
    textLayer.TextItem.HorizontalScale += 1
textLayer.TextItem.HorizontalScale -= 1

In [717]:
#boundsLayer = layer.parent.Layers["Card Name - Bounds"]
print(boundsLayer.bounds)
print(layer.bounds)#TextItem.

(184.0, 1420.0, 1177.0, 1635.0)
(185.0, 1421.0, 1182.0, 1624.0)


In [101]:
win32app = win32com.client.Dispatch("Photoshop.Application")
psd_api = win32app.Open(ygopsd.path)
"""
psd_api = ygopsd.PhotoshopApp.Open(ygopsd.path)
"""
layerName = "Effect Card Lore"
hierarchy = ygopsd.GetGroupOrLayerPathFromRootByName(layerName)
layer = psd_api
hierarchy.pop()
while (len(hierarchy) > 0) :
    childName = hierarchy.pop()
    #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
    layer = layer.Layers[childName]
#layer.TextItem.Contents = ygoCardInfo.df.loc["Le Renoncé"]["desc"]
#boundsLayer = layer.parent.Layers[layerName + " - Bounds"]
#CardTextDownLimit = boundsLayer.Bounds[3]

In [74]:
#boundsLayer.TextItem.HorizontalScale += 1
#layer.TextItem.Size = 14.5
layer.TextItem.Leading

4.908629264831543

In [76]:
"""layer.TextItem.Size = 10
limitFontSizeInUnit = layer.TextItem.Size"""
currentFontSize = 14.5
layer.TextItem.Leading = currentFontSize
layer.TextItem.Size = currentFontSize
layer.TextItem.HorizontalScale = 100
layer.TextItem.VerticalScale = 100
iteration = 0
while (layer.Bounds[3] > CardTextDownLimit) :
    if (currentFontSize > 10):
        currentFontSize -= .5
        layer.TextItem.Leading = currentFontSize
        layer.TextItem.Size = currentFontSize
    layer.TextItem.HorizontalScale -= 10
    iteration += 1
    print(f"### Iteration n°{iteration}")
    print(f"Size = {layer.TextItem.Size}")
    print(f"HorizontalScale = {layer.TextItem.HorizontalScale}")
    print(f"VerticalScale = {layer.TextItem.VerticalScale}")
while (layer.Bounds[3] <= CardTextDownLimit) :
    layer.TextItem.HorizontalScale += 1
layer.TextItem.HorizontalScale -= 1

### Iteration n°1
Size = 4.739365997314453
HorizontalScale = 90
VerticalScale = 100
### Iteration n°2
Size = 4.570102386474609
HorizontalScale = 80
VerticalScale = 100
### Iteration n°3
Size = 4.400839805603027
HorizontalScale = 70
VerticalScale = 100


In [161]:
ygopsd.IsLayerInBounds(layer, boundsLayer.bounds, [2])
textLayer = layer
boundaries = boundsLayer.bounds 
limit = [2]
for direction in limit :
    if (direction < 2) : #Left or Up
        if (layer.Bounds[direction] > boundaries[direction]) :
             print(False)
    elif (direction == 2) :               #Right or Down
        if (layer.Bounds[direction] < boundaries[direction]) :
             print(False)
print(True)

False

In [215]:
cardInfoToPsd.ygoPsd.PrintHierarchy()

-> [Root] X
	-> Bleed X
	-> Card Border 
	-> [Textures] X
		-> Normal Texture 
		-> Effect Texture X
		-> Ritual Texture 
		-> Fusion Texture 
		-> Synchro Texture 
		-> Dark Synchro Texture 
		-> Xyz Texture 
		-> Slifer Texture 
		-> Obelisk Texture 
		-> Ra Texture 
		-> Legendary Dragon Texture 
		-> Spell Texture 
		-> Trap Texture 
		-> Token Texture 
		-> [Link Texture With Text BG] 
			-> Link Texture 
			-> Link (Flavour BG) 
	-> [General] X
		-> [Text On Frame] X
			-> [Set Number] X
				-> Set Number (Pendulum) 
				-> Set Number (Link) 
				-> Set Number X
			-> Disclaimer 
			-> Copyright X
			-> Code X
			-> Edition X
			-> Illegal 
		-> Make Text On Frame White 
		-> Art Box X
		-> Art Box copy 
		-> Art Box (Anniversary) 
		-> Lore Background X
		-> Lore Border X
		-> Header X
		-> Texture Bevel X
		-> ArtBoxShade X
		-> [Card Name] X
			-> Card Name (Silver) 
			-> Card Name (Black) X
			-> Card Name (White) 
			-> Card Name (Gold) 
		-> Hologram X
	-> [Artwork] X
		->

In [85]:
layerName = "Effect Text"
hierarchy = ygopsd.GetGroupOrLayerPathFromRootByName(layerName)
layer = ygopsd.PhotoshopApp.Open(ygopsd.path)
hierarchy.pop()
while (len(hierarchy) > 0) :
    childName = hierarchy.pop()
    #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
    layer = layer.Layers[childName]
layer.TextItem.Contents = test

In [233]:
ygoCardInfo.GetCardInfoByName("Mausolée de l'Empereur")["desc"]

GetCardInfoByName(name = Mausolée de l'Empereur) -> Already in Database


"Durant la Main Phase : le joueur du tour peut activer ces effets.\n●Payez 1000 LP ; immédiatement après la résolution de cet effet, Invoquez Normalement 1 monstre depuis votre main qui nécessite 1 Sacrifice, sans Sacrifier.\n●Payez 2000 LP ; immédiatement après la résolution de cet effet, Invoquez Normalement 1 monstre depuis votre main qui nécessite 2 Sacrifices, sans Sacrifier.\n(C'est son Invocation/Pose Normale pour ce tour.)"

In [171]:
ygopsd = YuGiOhCardPsd(path=r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\TempYuGiOhTemplate.psd")
win32app = win32com.client.Dispatch("Photoshop.Application")
app = win32app#ygopsd.PhotoshopApp#
psd_api = app.Open(ygopsd.path)
"""
psd_api = ygopsd.PhotoshopApp.Open(ygopsd.path)
"""
layerName = "Type/Ability"
hierarchy = ygopsd.GetGroupOrLayerPathFromRootByName(layerName)
layer = psd_api
hierarchy.pop()
while (len(hierarchy) > 0) :
    childName = hierarchy.pop()
    #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
    layer = layer.Layers[childName]
#layer.TextItem.Contents = ygoCardInfo.df.loc["Le Renoncé"]["card_text"]
#boundsLayer = layer.parent.Layers[layerName + " - Bounds"]
#CardTextDownLimit = boundsLayer.Bounds[3]

DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = 'Bleed') -> 'Bleed' is already visible.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = 'Copyright') -> 'Copyright' is already visible.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = <LayerNames.SerialNumber: 'Code'>) -> 'Code' is already visible.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = <LayerNames.Edition: 'Edition'>) -> 'Edition' is already visible.


In [181]:
layer.TextItem.Contents# = layer.TextItem.Contents#.replace("Type", "Kek")

'[Type/Ability]'

In [801]:
boundingLayer = layer.parent.ArtLayers.Add()
boundingLayer.Name = layer.name + " - Bounds"

strokeColor = comtypes.client.CreateObject("Photoshop.SolidColor")
strokeColor.RGB.Red = 255
strokeColor.RGB.Green = 0
strokeColor.RGB.Blue = 255


SelectFromBoundaries(app, layer.bounds)
app.activeDocument.selection.Fill(strokeColor)
#app.activeDocument.selection.stroke(strokeColor, 4, , 2, 100)

In [9]:
from enum import Enum 

@unique
class ColorTest(Enum) :
    Black   = 1
    White   = 2
    Silver  = 3
    Gold    = 4

In [214]:
class YuGiOhPsdSetup :
    def __init__(self, 
                 path = r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\YuGiOhTemplate.psd", 
                 language = "fr", 
                 layerNameToInferCardBorders = "Fusion Texture",
                 ygoPsd = None) :
        self.language = language
        self.path = path
        self.maxSpread = 100
        if (ygoPsd is None) :
            self.ygoPsd = YuGiOhCardPsd(path = path)
        else :
            self.ygoPsd = ygoPsd
        self.cardBounds = self.GetLayerBounds(layerNameToInferCardBorders)
        self.app = self.ygoPsd.PhotoshopApp
        self.psd_api = self.app.Open(self.path)
        
    def GetLayerBounds(self, layerName) : 
        layer = ygopsd.GetGroupOrLayerByName(layerName)
        if (layer != None) :
            return layer.bbox
        else :
            return None
            
    def GetAnchorPointFromSpreadingDirection(self, spreadingDirection = (Direction.Bottom, Direction.Middle)) :
        topBottom = spreadingDirection[0]
        leftRight = spreadingDirection[1]
        
        if (topBottom == Direction.Bottom) :
            if (topBottom == Direction.Left) :
                return PsAnchorPosition.psTopRight
            elif (topBottom == Direction.Right) :
                return PsAnchorPosition.psTopLeft
            else :
                return PsAnchorPosition.psTopCenter
        elif (topBottom == Direction.Top) :
            if (topBottom == Direction.Left) :
                return PsAnchorPosition.psBottomRight
            elif (topBottom == Direction.Right) :
                return PsAnchorPosition.psBottomLeft
            else :
                return PsAnchorPosition.psBottomCenter
        else :
            if (topBottom == Direction.Left) :
                return PsAnchorPosition.psMiddleRight
            elif (topBottom == Direction.Right) :
                return PsAnchorPosition.psMiddleLeft
            else :
                return PsAnchorPosition.psMiddleCenter
            
    def SpreadLayer(self, layer, spreadingDirection = (Direction.Bottom, Direction.Middle), maxSizeSpread = 100) :
        hSize = layer.bounds[2] - layer.bounds[0]
        vSize = layer.bounds[3] - layer.bounds[1]
        hPerc = 100
        vPerc = 100
        
        topBottom = spreadingDirection[0]
        leftRight = spreadingDirection[1]
        
        if (leftRight != Direction.Middle) :
            hAdd = max(maxSizeSpread, abs(layer.bounds[leftRight] - self.cardBounds[leftRight]))
            hPerc = 100 * (hSize + hAdd) / hSize
        
        if (topBottom != Direction.Middle) :
            hAdd = max(maxSizeSpread, abs(layer.bounds[topBottom] - self.cardBounds[topBottom]))
            vPerc = 100 * (vSize + hAdd) / vSize
            
        textParams = layer.TextItem
        layer.Resize(Horizontal = hPerc,
                     Vertical = vPerc,
                     Anchor = self.GetAnchorPointFromSpreadingDirection(spreadingDirection))
        layer.TextItem = textParams
        
    def CreateBoundingBox(self,
                          textLayerName,
                          spreadingDirection = (Direction.Bottom, Direction.Middle),
                          colorInRGB = (255, 0, 255)) :
        hierarchy = self.ygoPsd.GetGroupOrLayerPathFromRootByName(textLayerName)
        layer = self.psd_api
        hierarchy.pop()
        while (len(hierarchy) > 0) :
            childName = hierarchy.pop()
            #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
            layer = layer.Layers[childName]
        boundingLayer = layer.parent.ArtLayers.Add()
        boundingLayer.Name = layer.name + " - Bounds"

        strokeColor = comtypes.client.CreateObject("Photoshop.SolidColor")
        strokeColor.RGB.Red   = colorInRGB[0]
        strokeColor.RGB.Green = colorInRGB[1]
        strokeColor.RGB.Blue  = colorInRGB[2]


        self.ygoPsd.SelectFromBoundaries(layer.bounds, self.app)
        self.app.activeDocument.selection.Fill(strokeColor)
        
        boundingLayer.visible = False
        
        return layer
        
    def CreateBoundingBoxAndSpread(self, 
                                   textLayerName, 
                                   spreadingDirection = (Direction.Bottom, Direction.Middle), 
                                   colorInRGB = (255, 0, 255)) :
        print("WARING : CreateBoundingBoxAndSpread(...) -> Not Implemented : Resizing Text bounding box changes text settings. Use CreateBoundingBox instead.")
        return False
        textLayer = self.CreateBoundingBox(textLayerName, spreadingDirection, colorInRGB)
        self.SpreadLayer(layer, spreadingDirection)
        #app.activeDocument.selection.stroke(strokeColor, 4, , 2, 100)
        

In [12]:
@unique
class PsAnchorPosition(IntFlag) :
    psTopLeft = 1
    psTopCenter = 2
    psTopRight = 3
    psMiddleLeft = 4
    psMiddleCenter = 5
    psMiddleRight = 6
    psBottomLeft = 7
    psBottomCenter = 8
    psBottomRight = 9

In [223]:
ygoPsdSetup = YuGiOhPsdSetup(path=r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\YuGiOhTemplate.psd")

DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = 'Bleed') -> 'Bleed' is already visible.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = 'Copyright') -> 'Copyright' is already visible.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = <LayerNames.SerialNumber: 'Code'>) -> 'Code' is already visible.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = <LayerNames.Edition: 'Edition'>) -> 'Edition' is already visible.
DEBUG : YuGiOhCardPsd.ChangeTextOnLayer(layerName = <LayerNames.Edition: 'Edition'>, text = 'ÉDITION LIMITÉE') -> setting value for Edition => 'ÉDITION LIMITÉE'.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = <LayerNames.Edition: 'Edition'>) -> 'Edition' is already visible.


In [224]:
for size in ["S", "M", "L"] :
    ygoPsdSetup.CreateBoundingBox(f"[{size}] Monster Effect (Pendulum)")
    ygoPsdSetup.CreateBoundingBox(f"[{size}] Monster Description (Pendulum)")
    ygoPsdSetup.CreateBoundingBox(f"[{size}] Pendulum Effect")

In [221]:
ygoPsdSetup.CreateBoundingBox(LayerNames.SpellTrapEffect)

<COMObject <unknown>>

In [935]:
level = 5
isRank = True

ygopsd = YuGiOhCardPsd(path=r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\TempYuGiOhTemplate.psd")

hierarchy = ygopsd.GetGroupOrLayerPathFromRootByName("Level/Rank")
levelGroup = psd_api
hierarchy.pop()
while (len(hierarchy) > 0) :
    childName = hierarchy.pop()
    #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
    levelGroup = levelGroup.Layers[childName]

try :
    levelGroup.ArtLayers.Remove(levelGroup.Layers["Current Levels"])
except :
    print("SetLevelOrRank(...) : no layer named 'Current Levels'")


if (not isRank) :
    layer = levelGroup.Layers["Level"]
else :
    layer = levelGroup.Layers["Rank"]
    
app.ActiveDocument.ActiveLayer = layer.Duplicate()

app.ActiveDocument.ActiveLayer.Name = "Current Levels"
maxLevel = 12

shift = 0
"""if (not isRank and level >= 6) :
    shift = -1
elif (isRank and level >= 9) :
    shift = 1
elif (isRank and level <= 2) :
    shift = -1"""

#if (level < 11) :
boundaries = app.ActiveDocument.ActiveLayer.bounds
if (level == 0) :
    newBoundaries = boundaries
elif (not isRank) :
    whereToCut = boundaries[0] + ((boundaries[2] - boundaries[0]) * (1 - level / maxLevel)) + shift
    newBoundaries = (boundaries[0], boundaries[1], whereToCut, boundaries[3])
else :
    whereToCut = boundaries[2] - ((boundaries[2] - boundaries[0]) * (1 - level / maxLevel)) + shift
    newBoundaries = (whereToCut, boundaries[1], boundaries[2], boundaries[3])
ygopsd.SelectFromBoundaries(newBoundaries)
app.ActiveDocument.Selection.Clear()

In [8]:
for pos in PsAnchorPosition :
    print(f"{pos.name} : {pos.value}")

psTopLeft : 1
psTopCenter : 2
psTopRight : 3
psMiddleLeft : 4
psMiddleCenter : 5
psMiddleRight : 6
psBottomLeft : 7
psBottomCenter : 8
psBottomRight : 9


In [17]:
MandatoryLayers = {
    "Card Name" : {
        "layerName" : "Card Name (Black)",
        "isBounded" : True,
        "layerNameFromColor" : (lambda x : f"Card Name ({x})"),
        "allowedColors" : ["Black", "White", "Gold"]#, "Silver"] 
    },
    "Monster Effect" : {
        
    }
}

In [18]:
MandatoryLayers["Card Name"]["getColor"]("Black")

'Card Name (Black)'

In [2]:
@unique
class ComponentTypes(Enum) :
    Layer = 1
    MetaLayer = 2
    
    Group = 10

In [3]:
class PsdLayer :
    def __init__(self,
                 layerName,
                 isInPsd = None) :#,
                  #isGroup = False) :
        self.layerName = layerName
        self.isInPSD   = isInPsd
        
    def GetFromPSD(self,
                psdManager) :
        if (self.isInPSD is False) :
            return None
        
        result = psdManager.GetGroupOrLayerByName(self.layerName)
        if (result is None) :
            self.isInPSD = False
        else :
            self.isInPSD = True
            
        return result

In [4]:
class PsdGroup(PsdLayer) :
    def __init__(self,
                 layerName,
                 isInPsd = None,
                 someLayersInPsd = None,
                 childs = []) :
        self.layerName       = layerName
        self.isInPSD         = isInPsd
        self.someLayersInPsd = someLayersInPsd
        self.childs          = childs
    
    def AddChild(self, componentType = ComponentTypes.Layer, **initArguments) :
        if (componentType == ComponentTypes.Layer) :
            self.childs.append(PsdLayer(initArguments))
        elif (componentType == ComponentTypes.Layer) :
            self.childs.append(PsdLayer(initArguments))
        else :
            print(f"ERROR -> PsdGroup.AddChild(componentType = '{componentType}') : Component type non supported.")

In [8]:
ygopsd = YuGiOhCardPsd(path=r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\TempYuGiOhTemplate.psd")
#ygopsd.PrintHierarchy()

NameError: name 'YuGiOhCardPsd' is not defined

In [3]:
from

In [30]:
class test(StrEnum) :
    test0 = "Kek"
    
    @classmethod
    def test1(self) :
        return self.test0.value

In [52]:
LayerNames.GetFrameLayerNameFromCardType(CardType.MonsterFusion)

<LayerNames.FusionMonsterFrame: 'Fusion Texture'>

In [74]:
value = "  0400"
replacementValueForACharacter = "|  |"#"  "
numberOfDigits = 4
if (type(value) == int) :

In [63]:
psdPath = r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\PSDs\Le_Renonc.psd"
pngPath = r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\RenonceTest.png"
win32app = win32com.client.Dispatch("Photoshop.Application")
app = win32app#ygopsd.PhotoshopApp#
psd_api = app.Open(psdPath)

In [64]:
options = win32com.client.Dispatch("Photoshop.ExportOptionsSaveForWeb")
options.Format = 13   # PNG Format
options.PNG8 = False  # Sets it to PNG-24 bit
options.Quality = 100

psd_api.Export(ExportIn=pngPath, ExportAs=2, Options=options)
psd_api.Close()

In [131]:
import requests
from bs4 import BeautifulSoup

def trouver_nom_anglais(nom_carte_francais):
    # Construire l'URL avec le nom de la carte en français
    url = f'https://www.otk-expert.fr/yugioh/cartes/{nom_carte_francais}/'

    # Faire la requête HTTP pour obtenir le contenu HTML de la page
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erreur lors de la requête HTTP. Code de statut : {response.status_code}")
        return None

    # Utiliser BeautifulSoup pour analyser le contenu HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Trouver le nom anglais de la carte dans le code HTML
    nom_anglais_tag = soup.find('div', {'class': 'col_middle'})
    if nom_anglais_tag:
        nom_anglais = nom_anglais_tag.h2.find_all('span')[1].text.strip()
        return nom_anglais
    else:
        print("Balise non trouvée dans le code HTML.")
        return None

# Test de la fonction avec le nom de carte "nuit-des-machines"
nom_carte_francais_test = "sp-petite-chevaleresse4"
nom_anglais_resultat = trouver_nom_anglais(nom_carte_francais_test)

# Affichage du résultat
if nom_anglais_resultat:
    print(f"Le nom anglais de la carte '{nom_carte_francais_test}' est : {nom_anglais_resultat}")
else:
    print("Échec de la récupération du nom anglais.")


ConnectionError: HTTPSConnectionPool(host='www.otk-expert.fr', port=443): Max retries exceeded with url: /yugioh/cartes/sp-petite-chevaleresse4/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C1A9918490>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [125]:
ygopsd = YuGiOhCardPsd()

DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = 'Bleed') -> 'Bleed' is already visible.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = 'Copyright') -> 'Copyright' is already visible.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = <LayerNames.SerialNumber: 'Code'>) -> 'Code' is already visible.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = <LayerNames.Edition: 'Edition'>) -> 'Edition' is already visible.
DEBUG : YuGiOhCardPsd.ChangeTextOnLayer(layerName = <LayerNames.Edition: 'Edition'>, text = 'ÉDITION LIMITÉE') -> setting value for Edition => 'ÉDITION LIMITÉE'.
DEBUG : YuGiOhCardPsd.ChangeGroupOrLayerVisibility(name = <LayerNames.Edition: 'Edition'>) -> 'Edition' is already visible.


In [217]:
win32app = win32com.client.Dispatch("Photoshop.Application")
comtypesapp = comtypes.client.CreateObject("Photoshop.Application", dynamic = True)
app = win32app#comtypesapp#
psd_api = app.Open(ygopsd.path)

pendulumSize = "M"
picturePath = r"C:\Users\dodoa\Pictures\MTG\Yu-Gi-Oh Proxies\testArtwork.jpg"

if (pendulumSize is not None) :
    pendulumSizeAsString = f"[{pendulumSize}] "
else :
    pendulumSizeAsString = ""          

hierarchy = ygopsd.GetGroupOrLayerPathFromRootByName(pendulumSizeAsString + LayerNames.PendulumSpellEffect)
layer = psd_api
hierarchy.pop()
while (len(hierarchy) > 0) :
    childName = hierarchy.pop()
    #print(f"Poping '{childName}' -> len = {len(hierarchy)}")
    layer = layer.Layers[childName]

In [222]:
layer.

(264.0, 1335.0, 1212.0, 1399.0)

In [32]:
test = "Lorem Ipsum"

In [126]:
activeArrows = [LinkDirections.TopLeft, LinkDirections.Top, LinkDirections.Left]
layerNames = []
for aa in activeArrows :
    layerNames.append(LayerNames.GetLinkArrowLayerNameFromEnum(aa))
arrowsGroup = ygopsd.GetGroupOrLayerByName(LayerNames.LinkArrowsGroup)
for arrow in arrowsGroup :
    isActive = arrow.name in layerNames
    for activeOrNot in arrow :
        activeOrNot.visible = ((activeOrNot.name == "Active") == isActive)
ygopsd.psd.save(ygopsd.path)
        

In [108]:
test = layer.Layers["Center"]

In [110]:
app.ActiveDocument.ActiveLayer = test

In [194]:
cardType = cardInfo["type"]
pendulumStringLowercase = "pendulum".lower()
if (pendulumStringLowercase in cardType.lower()) :
    startIndex = cardType.lower().index(pendulumStringLowercase)
    newCardType = ""
    if (startIndex > 0) :
        newCardType = cardType[:startIndex].strip()
    if ((startIndex + len(pendulumStringLowercase)) < (len(cardType) - 1)) :
        lastPart = cardType[startIndex + len(pendulumStringLowercase):].strip()
        newCardType = f"{newCardType.strip()} {lastPart.strip()}".strip()    
newCardType

'Effect Monster'

{'linkval': None,
 'race': 'Machine',
 'atk': 1000,
 'name_en': 'Qliphort Scout',
 'attribute': 'EARTH',
 'def': 2800,
 'frameType': 'normal_pendulum',
 'level': 5,
 'type': 'Pendulum Normal Monster',
 'desc': '[ Pendulum Effect ] \nYou cannot Special Summon monsters, except "Qli" monsters. This effect cannot be negated. Once per turn: You can pay 800 LP; add 1 "Qli" card from your Deck to your hand, except "Qliphort Scout".\n[ Monster Effect ] \nDémarrage en Mode Réplique...\nUne erreur est survenue lors de l\'exécution de C:\\sophia\\zefra.exe\nÉditeur inconnu.\nAutoriser C:\\tierra\\qliphort.exe ? ＜O／N＞...[O]\nDémarrage en Mode Autonomie…',
 'name': 'Sentinelle Qliphort',
 'id': 65518099}

In [187]:
cardType.lower().index("pendulum")

0

In [326]:
test = YGOProDeckCardInfos()
kek = test.GetCardInfoByName("Mausolée de l'empereur")

{'linkval': None, 'pendulum_scale': None, 'race': 'Field', 'atk': None, 'name_en': 'Mausoleum of the Emperor', 'attribute': None, 'def': None, 'frameType': 'spell', 'level': None, 'type': 'Spell Card', 'pendulum_effect': None, 'desc': "Durant la Main Phase : le joueur du tour peut activer ces effets.\n●Payez 1000 LP ; immédiatement après la résolution de cet effet, Invoquez Normalement 1 monstre depuis votre main qui nécessite 1 Sacrifice, sans Sacrifier.\n●Payez 2000 LP ; immédiatement après la résolution de cet effet, Invoquez Normalement 1 monstre depuis votre main qui nécessite 2 Sacrifices, sans Sacrifier.\n(C'est son Invocation/Pose Normale pour ce tour.)", 'name': "Mausolée de l'Empereur", 'id': 80921533}
{'linkval': None, 'pendulum_scale': None, 'race': 'Field', 'atk': None, 'name_en': 'Mausoleum of the Emperor', 'attribute': None, 'def': None, 'level': None, 'type': 'Spell Card', 'pendulum_effect': None, 'name': "Mausolée de l'Empereur", 'id': 80921533, 'frame_type': 'spell', 

In [132]:
name = "Cylindre Magique"
cardInfo = cardInfoToPsd.ygoCardInfo.GetCardInfoByName(name)

GetCardInfoByName(name = Cylindre Magique) -> Already in Database


In [133]:
cardInfo

id                                                          62279055
card_textdesc                                                    NaN
race                                                        [Normal]
cardTypeEnum                                                     NaN
type                                                            Trap
name_en                                               Magic Cylinder
atk                                                             None
def                                                             None
level                                                           None
attribute                                                       None
artworksUrls       [https://images.ygoprodeck.com/images/cards_cr...
linkval                                                         None
linkmarkers                                                     None
pendulum_effect                                                 None
pendulum_scale                    

In [189]:
f"{qty} :: {name}"

'1 :: Merveilleux Ballons'